In [2]:
load("../framework/instance_gen.sage")
load("../framework/geometry_twist.sage")
load("../framework/geometry.sage")
demo = False
from scipy.io import loadmat
import random

In [3]:
def set_parameters(params):
    #for params in [ 'CCS1']:#, 'CCS2', 'CCS3', 'CCS4', 'NIST1', 'NIST2']:
    logging("Set of parameters: " + params)
    if params == 'NIST1':
        # NIST1 FRODOKEM-640
        n = 640
        m = 640
        q = 2**15
        frodo_distribution = [9456, 8857, 7280, 5249, 3321,
                              1844, 898, 384, 144, 47, 13, 3]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        av_prev, var_prev = average_variance(D_s)
        # We used the following seeds for generating Bos et al. data
        # These seeds were generated with the matlab code genValues.m
        sca_seeds = [42, 72, 163, 175, 301, 320, 335, 406, 430, 445]
        param = 4

    elif params == 'NIST2':
        # NIST2 FRODOKEM-976
        n = 976
        m = 976
        q = 65536
        frodo_distribution = [11278, 10277, 7774, 4882, 2545, 1101,
                              396, 118, 29, 6, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        av_prev, var_prev = average_variance(D_s)
        sca_seeds = [74, 324, 337, 425, 543, 1595, 1707, 2026, 2075, 2707]
        param = 5

    elif params == 'CCS1':
        n = 352
        m = 352
        q = 2 ** 11
        frodo_distribution = [22528, 15616, 5120, 768]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        av_prev, var_prev = average_variance(D_s)
        sca_seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        param = 0

    elif params == 'CCS2':
        n = 592
        m = 592
        q = 2 ** 12
        frodo_distribution = [25120, 15840, 3968, 384, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        av_prev, var_prev = average_variance(D_s)
        sca_seeds = [2, 3, 4, 5, 6, 7, 8, 9, 14, 16]
        param = 1


    elif params == 'CCS3':
        n = 752
        m = 752
        q = 2 ** 15
        frodo_distribution = [19296, 14704, 6496, 1664, 240, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        av_prev, var_prev = average_variance(D_s)
        sca_seeds = [2, 4, 5, 7, 9, 10, 12, 13, 14, 15]
        param = 2

    elif params == 'CCS4':
        n = 864
        m = 864
        q = 2 ** 15
        frodo_distribution = [19304, 14700, 6490, 1659, 245, 21, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        av_prev, var_prev = average_variance(D_s)
        sca_seeds = [256, 393, 509, 630, 637, 652, 665, 1202, 1264, 1387]
        param = 3
    else:
        print("Wrong parameters")
        return
    return n, m, q, frodo_distribution, D_s, av_prev, var_prev, sca_seeds, param

In [4]:
def data_to_measure(sca_seeds, params, param, frodo_distribution): 
    # Reading the score tables from Bos et al. attack
    scores = []
    correct = []
    for seed in sca_seeds:
        for i in range(1, 9):
            data = loadmat('Scores_tables_SCA/' + params +
                           '/apostandcorrect' + str(param + 1) +
                           '_seed' + str(seed) +
                           'nbar' + str(i) + '.mat')
            scores += list(data['apostdist'])
            correct += list(data['correct'])
    measures = {}
    # the score tables are stored according to the secret coefficient. We use them
    # for generating the measurement.
    for key_guess in range(-len(frodo_distribution)+1, len(frodo_distribution)):
        measures[key_guess] = [scores[ind] for ind in
                  [i for i, d in enumerate(correct) if
                   recenter(d) == key_guess]]
    return measures
def simu_measured(secret, measures):
    """
    This fonction simulates the information gained by
    Bos et al attack. For a given secret, the simulation
    is outputting a score table at random from the data of Bos et al.
    :secret: an integer being the secret value
    :measurement: a score table
    """
    if secret in measures.keys():
        measurement = random.choice(measures[secret])
    else: 
        measurement = None
    return measurement

def measurement_to_aposteriori(measurement):
    """
    This fonction transforms a score table into an aposteriori distribution.
    According to the matlab code of Bos et al. attack,
    the score table is proportional
    to the logarithm of the aposteriori probability. We thus apply the exponential
    and renormalize.
    :measurement: a score table
    :apost_dist: a dictionnary of the aposteriori distribution
    """
    if measurement is not None:
        s = sum([exp(meas) for meas in measurement])
        measurement = [exp(meas)/s for meas in measurement]
        apost_dist = {}
        for key_guess in range(-len(frodo_distribution) + 1, len(frodo_distribution)):
            apost_dist[key_guess] = measurement[key_guess + len(frodo_distribution) - 1]
    else:
        apost_dist = None
    return apost_dist


In [5]:
def count_SCA_thres(var_prev, measured, thres_prob):
    """ 
    This function evaluates the security loss after Bos et al attack
    :report_every: an integer that give the frequency of
    logging (None for no logging)
    :dbdd: instance of the class DBDD
    :measured: table representing the (simulated) information
    given by Bos et al attack
    :max_guesses: integer for upperbounding the number of guesses

    :s_mu: mean vector of s in posteriori distribution
    :s_var: variance of s in posteriori distribution
    """
    proba_best = {}
#         Id = identity_matrix(n + m)
#         num = 0
    num_guess = 0
    proba_success = 1.
    guesses = 0 
    list_av = [0] * n
    list_var = [0] * n
    gap_list = []
    # index of 1-dim ellipsoid of side-channel distribution is not fully contained in the original distirbution
    intersect_indices = []
#     tangent_indices = dict()
    guess_indices = []
    for i in range(n):
        apost_dist = measurement_to_aposteriori(measured[i])
        if apost_dist is not None:
            proba_best[i] = max([apost_dist[j] for j in apost_dist.keys()])
            av, var = average_variance(apost_dist)
            av = float(av)
            var = float(var)
            list_av[i] = av
            list_var[i] = var

#             print("av, var, var_prev", av, var, var_prev)
            if av*av > var_prev - var:
#                 print("diff: ", av*av - (var_prev - var))
#                 print("include or not: ", av*av - (var - var_prev))
                gap_list += [av*av - (var_prev - var)] #gap is how much posterior ellipsoid exceeds prior ellipsoid
                intersect_indices += [i]
#                 if av*av - var < var_prev:
#                     print("-")
#                 if av*av - var >= var_prev: 
# #                     print("tangent")
#                     tangent_indices[i] = av*av - var
# #                     print("av, sqrt var, sqrt var_prev", av, sqrt(var), sqrt(var_prev))
# #                     num += 1
# #                     if proba_best[i] < thres_prob:
# #                         num_guess += 1
            if proba_best[i] > thres_prob: 
                proba_success *= proba_best[i]
                
                guesses += 1
                guess_indices += [i]

        else:
            av = None
            var = None
            print("None in av and var, index: ", i)
#     print("s_av, s_var: ", list_av, list_var)
    return list_av, list_var, intersect_indices, proba_success, guesses, guess_indices, gap_list


In [6]:
#contains update mean
def posteriori_integration_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, n, m, factor):
    print("scaled@funct posteriori_integration?: ", ebdd.scaled)
    list_std_inv = [1/sqrt(list_var[i]) for i in range(len(list_var))]

    B_post_sub = diagonal_matrix(list_std_inv)
    c_post_sub = vec(list_av)
    
#     ####update mean#####
#     zero1 = zero_matrix(ZZ, m, n)
#     upleft_mat = block4(identity_matrix(n), -A.T, zero1, q*identity_matrix(m))
#     b_new = b - c_post_sub * A.T
#     b0 = concatenate(zero_matrix(QQ, 1, n), -b_new)
#     upleft_mat_inv = upleft_mat.inverse()
#     mu = b0 * upleft_mat_inv
#     c_post = concatenate(c_post_sub, mu[0][n:])
#     ebdd.mu = c_post
#     ####update mean#####
    

    
    c_post = mean_s_c_update(A, b, m, n, c_post_sub)
    ebdd.mu = c_post
#     print("c_post_sub: ", c_post_sub[0, :])
#     print("ebdd.mu: ", ebdd.mu[0, :])

    B_post = block4(B_post_sub, B_Sigma[0:n, n:], B_Sigma[n:, 0:n], B_Sigma[n:, n:])
    
     #concatenate(c_post_sub, mu[0][n:])
    
    W_SCA_full = B_post*B_post.T * 1/(n + m)
    S1 = matrix(np_inv(W_SCA_full)) #<= 1
    S1_extend = S1 + c_post.T * c_post*factor
    
#     ebdd.mu = mu #concatenate(mu_s, mu[0][n:])
    ebdd.S = S1_extend / (n + m)#matrix(np_inv(B_post*B_post.T))    <= dim
    print("scaled or not suppose false: ",ebdd.scaled)
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - ebdd.mu)).T))
#     print("ellipsoid norm of secret after posteriori_integration2: ", scal((ebdd.u - c_post2) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - c_post2)).T))
    
    return c_post, B_post

In [7]:
#contains update mean
#used in first guess then intersect
def hints_integration_Sigma(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, guess_indices, m, n, factor, D_s):
    """
    Increase LWE ellipsoid in the bad coordinates to ensure the intersections
    B_SCA, c_SCA--posteriori
    B_LWE, c_LWE--prior
    
    """
    
#     print("uuuuuuu@hints", ebdd.u[0][n - 5 :n + 5])
    print("muuuuuu@hints", ebdd.mu[0][n - 5 :n + 5])
    print("vec(list_av)@hints", vec(list_av)[0][n - 5 :n + 5])
#     print("muuuuuu@hints", ebdd.mu[0])
#     print("vec(list_av)@hints", vec(list_av)[0])
    
#     print("B_Sigma ellipsoid norm: ", scal((ebdd.u - ebdd.mu) * B_Sigma * B_Sigma.T/(m + n) * ((ebdd.u - ebdd.mu)).T))
    
    print("----------------------")
    print("ebdd.S@hints_integration_Sigma: ", matrix(RR,ebdd.S[n - 2 :n + 3,n - 2 :n + 3]), ebdd.S.rank())
    S_SCA = diagonal_matrix(list_var)*n #<= 1
    
    c_SCA = vec(list_av)
    print("len of mu_guess", ebdd.mu.ncols())
    print("expecting False, not scaled: ", ebdd.scaled)
    
    ##LWE:
    S_LWE = matrix(S_SCA)
    c_LWE = matrix(c_SCA)
    print("VOL OF ELLIP_LWE: ", -ln(S_LWE.det()).n())
    mu_s, s_s = average_variance(D_s)
    
    for i in intersect_indices:
#         print("prev B_LWE[i, i], c_LWE[0, i]", S_LWE_dim[i, i], c_LWE[0, i])
        S_LWE[i, i] = s_s*n #<=1
        c_LWE[0, i] = mu_s
        print(i, "VOL OF ELLIP_LWE: ", -ln(S_LWE.det()).n(), S_LWE.rank(), (S_LWE.T).rank())
#         print("B_LWE[i, i], c_LWE[0, i]", S_LWE_dim[i, i], c_LWE[0, i])

    print("VOL OF ELLIP_LWE: ", -ln(S_LWE.det()).n())
    print("VOL OF ELLIP_SCA: ", -ln(S_SCA.det()).n())
    
    mu_s, Sigma_s = ellipsoid_intersection_Sigma(c_LWE, S_LWE, c_SCA, S_SCA, tolerance=1e-10) #<= 1
    print("VOL OF ELLIP_OUTPUT: ", ln(Sigma_s.det()).n())
    
#     ####update mean#####
#     zero1 = zero_matrix(ZZ, m, n)
#     upleft_mat = block4(identity_matrix(n), -A.T, zero1, q*identity_matrix(m))
#     b_new = b - mu_s * A.T
#     b0 = concatenate(zero_matrix(QQ, 1, n), -b_new)
#     upleft_mat_inv = upleft_mat.inverse()
#     mu = b0 * upleft_mat_inv
#     ebdd.mu = concatenate(mu_s, mu[0][n:])
#     ####update mean#####
    print("dimension match check @hints", mu_s.ncols(), n, A.ncols())
    print("dimension match check @hints", b.ncols(), m, A.nrows())
    mu = mean_s_c_update(A, b, m, n, mu_s)  
    ebdd.mu = mu
    
    Sigma_s_inv = matrix(np_inv(Sigma_s * 1/n)) # convert to <= dim
    B_new_sub = matrix(scipy.linalg.cholesky(Sigma_s_inv)) #lower=True

    B_hybrid_eps = block4(B_new_sub, zero_matrix(ZZ, n, m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, m, m))
    E_hybrid_eps = B_hybrid_eps * B_hybrid_eps.T * (1/n) #<= 1
    
    
    B_Sigma_eps = block4(zero_matrix(ZZ, n, n), B_Sigma[0:n, n:], B_Sigma[n:, 0:n], B_Sigma[n:, n:]) #<=dim
    
    new_S_inv = (n)/(m + n) * E_hybrid_eps + (1 - (n)/(m + n)) * (1/(m))*B_Sigma_eps*B_Sigma_eps.T    #<= 1          
    ebdd.S = matrix(np_inv(new_S_inv * (n + m))) #<= dim
    print("ellip_norm: ")
    print(scal((ebdd.u - ebdd.mu) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - ebdd.mu)).T))
    return mu_s, Sigma_s, ebdd.S
    


In [8]:
# def main(params, nb_tests_per_params, thres_prob):
"""  Averaging
The following averages the measures to get accurate data
for the paper. The averaging mode is quite long.
"""
params = 'CCS1'

eps = 10**-10
n, m, q, frodo_distribution, D_s, av_prev, var_prev, sca_seeds, param = set_parameters(params)


 Set of parameters: CCS1 


352

In [270]:
load("../framework/instance_gen.sage")
load("../framework/geometry_twist.sage")
load("../framework/geometry.sage")
demo = False
from scipy.io import loadmat
import random

In [9]:
nb_tests_per_params = 1
thres_prob = 0.995

A_list = [0]*nb_tests_per_params
b_list = [0]*nb_tests_per_params
e_vec_list = [0]*nb_tests_per_params
ebdd_list = [0]*nb_tests_per_params
B_Sigma_list = [0]*nb_tests_per_params
measured_list = [0]*nb_tests_per_params


A2_list = [0]*nb_tests_per_params
b2_list = [0]*nb_tests_per_params
e_vec2_list = [0]*nb_tests_per_params
ebdd2_list = [0]*nb_tests_per_params
B_Sigma2_list = [0]*nb_tests_per_params
measured2_list = [0]*nb_tests_per_params

for ind in range(nb_tests_per_params):
    print("Sample ", ind)
    ####################Initial Phase#######################
    A, b, e_vec, ebdd, B_Sigma = initialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo,
                                                                   n,
                                                                   q, m, D_s, D_s, homogeneous=False, verbosity=1)
    
    A_list[ind] = matrix(A)
    b_list[ind] = matrix(b)
    e_vec_list[ind] = matrix(e_vec)
    
    B1_temp = matrix(ebdd.B)
    S1_temp = matrix(ebdd.S)
    mu1_temp = matrix(ebdd.mu)
    u1_temp = matrix(ebdd.u)
    ebdd_list[ind] = EBDD_non_homo(B1_temp, S1_temp, mu1_temp, u1_temp)
    
    B_Sigma_list[ind] = B_Sigma
    
    
    ###
    A2 = matrix(A)
    b2 = matrix(b)
    e_vec2 = matrix(e_vec)
    s2 = matrix(ebdd.u[:, :n])
    _, _, _, ebdd2, B_Sigma2 = initialize_kannan_ellipsoid_from_LWE_derandom(EBDD_non_homo,
                                                                              n, q, m, D_s, D_s, 
                                                                              homogeneous=False, verbosity=1,
                                                                              A = A2, s = s2, e_vec = e_vec2)
    
    
    A2_list[ind] = A2
    b2_list[ind] = b2
    e_vec2_list[ind] = e_vec2
    
    B2_temp = matrix(ebdd2.B)
    S2_temp = matrix(ebdd2.S)
    mu2_temp = matrix(ebdd2.mu)
    u2_temp = matrix(ebdd2.u)
    ebdd2_list[ind] = EBDD_non_homo(B2_temp, S2_temp, mu2_temp, u2_temp)
    
#     ebdd2_list[ind] = EBDD_non_homo(ebdd2)
    B_Sigma2_list[ind] = B_Sigma2
    B_Sigma3 = matrix(B_Sigma)
    print(B_Sigma3 == B_Sigma2)
    
    
    measures = data_to_measure(sca_seeds, params, param, frodo_distribution)
    measured = [simu_measured(ebdd.u[0, i], measures) for i in range(0, n)]
    
    measured_list[ind] = measured
                                                                          
    ###                                                                      
    measured2 = list(measured)
    ###    
    measured2_list[ind] = measured2

Sample  0
      Build EBDD from LWE      
 n=352 	 m=352 	 q=2048 
ellipnorm of secret@initialize:  1.02056277056277
      Build EBDD from LWE      
 n=352 	 m=352 	 q=2048 
ellipnorm of secret@initialize_derandom:  1.02056277056277
True


In [166]:
print(B_Sigma == B_Sigma2)

True


In [13]:
#guess_then_intersect
for ind in range(nb_tests_per_params):
    A = A_list[ind]
    print(A.ncols())
    b = b_list[ind]
    e_vec = e_vec_list[ind]
    ebdd = ebdd_list[ind]
    B_Sigma = B_Sigma_list[ind]
    measured = measured_list[ind]
#     beta_origin, beta_post, beta_guess, beta_intersect, beta_short, dvol_origin, dvol_post, dvol_guess, dvol_intersect, dvol_short, p_success, guesses, list_cov_prev, list_cov_curr = guess_then_intersect(A, b, e_vec, ebdd, B_Sigma, measured)
    A1_guess, b1_guess, e1_vec, q, short_mu1_s, short_mu1_e, ebdd1_guess = guess_then_intersect(A, b, e_vec, ebdd, B_Sigma, measured)

352
1 scaled to <= 1? False
       Attack Estimation      
 dim=704 	 δ=1.005205 	 β=269.08  
  
########initial beta########:  269.0762395251977
1.5 scaled to <= 1? False
to check here is scaled
VOL OF ELLIP at Initial Phase:  -5176.29043055573
scaled@funct posteriori_integration?:  False
None in mean_e@mean_s_c_update
scaled or not suppose false:  False
ellipsoid norm of secret after posteriori_integration:  0.8803137381115889
expect false:  False
ellipsoid norm of secret after posteriori_integration:  0.8803137381115889
ellipsoid norm of secret after posteriori_integration:  0.8803137382163619
       Attack Estimation      
 dim=704 	 δ=1.007146 	 β=161.51  
  
########beta after only integrating posteriori distribution######## 161.51177477315065
to check here is scaled
VOL OF ELLIP after Posteriori Phase:  -infinity
scaled? expect <= dim here False
      Build chopped EBDD from previous LWE      
----------------------
Sigma_unscaled@reinitial:  [ 0.0000763000000000000      0.00000

87 VOL OF ELLIP_LWE:  -infinity 183 183
90 VOL OF ELLIP_LWE:  -infinity 183 183
91 VOL OF ELLIP_LWE:  -infinity 183 183
92 VOL OF ELLIP_LWE:  -infinity 183 183
93 VOL OF ELLIP_LWE:  -infinity 183 183
94 VOL OF ELLIP_LWE:  -infinity 183 183
95 VOL OF ELLIP_LWE:  -infinity 183 183
96 VOL OF ELLIP_LWE:  -infinity 183 183
97 VOL OF ELLIP_LWE:  -infinity 183 183
98 VOL OF ELLIP_LWE:  -infinity 183 183
99 VOL OF ELLIP_LWE:  -infinity 183 183
101 VOL OF ELLIP_LWE:  -infinity 183 183
102 VOL OF ELLIP_LWE:  -infinity 183 183
103 VOL OF ELLIP_LWE:  -infinity 183 183
104 VOL OF ELLIP_LWE:  -infinity 183 183
105 VOL OF ELLIP_LWE:  -infinity 183 183
106 VOL OF ELLIP_LWE:  -infinity 183 183
107 VOL OF ELLIP_LWE:  -infinity 183 183
108 VOL OF ELLIP_LWE:  -infinity 183 183
110 VOL OF ELLIP_LWE:  -infinity 183 183
111 VOL OF ELLIP_LWE:  -infinity 183 183
113 VOL OF ELLIP_LWE:  -infinity 183 183
116 VOL OF ELLIP_LWE:  -infinity 183 183
117 VOL OF ELLIP_LWE:  -infinity 183 183
118 VOL OF ELLIP_LWE:  -inf

/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:543: RuntimeWarning: invalid value encountered in log


(Solution) pi = 0.680712260240249
alpha = 0.218745095815443
X*X_inv MSE = 8.097313630437542e-36
VOL OF ELLIP_OUTPUT:  442.061256029742
dimension match check @hints 183 183 183
dimension match check @hints 352 352 352
None in mean_e@mean_s_c_update
ellip_norm: 
1.8037520697185592
       Attack Estimation      
 dim=535 	 δ=1.009239 	 β=100.53  
  
2 ebdd_guess scaled to <= 1? False
############beta after integrating SCA info########### 100.53329544307465
expect false:  False
ellipsoid norm of secret after hints_integration:  1.3707490870730616
to check here is scaled
VOL OF ELLIP at Intersect Phase:  -infinity


In [16]:
#intersect_then_guess
for ind in range(nb_tests_per_params):
    A2 = A2_list[ind]
    b2 = b2_list[ind]
    e_vec2 = e_vec2_list[ind]
    ebdd2 = ebdd2_list[ind]
    B_Sigma2 = B_Sigma2_list[ind]
    measured2 = measured2_list[ind]
#     beta_origin, beta_post, beta_guess, beta_intersect, beta_short, dvol_origin, dvol_post, dvol_guess, dvol_intersect, dvol_short, p_success, guesses, list_cov_prev, list_cov_curr = intersect_then_guess(A2, b2, e_vec2, ebdd2, B_Sigma2, measured2)
    A2_guess, b2_guess, e2_vec, q, short_mu2_s, short_mu2_e, ebdd2_guess = intersect_then_guess(A2, b2, e_vec2, ebdd2, B_Sigma2, measured2)

      intersect_then_guess      
1 scaled to <= 1? False
       Attack Estimation      
 dim=704 	 δ=1.007146 	 β=161.51  
  
########initial beta########:  161.51177477315065
1.5 scaled to <= 1? False
to check here is scaled
VOL OF ELLIP at Initial Phase:  -infinity
scaled@funct posteriori_integration?:  False
None in mean_e@mean_s_c_update
scaled or not suppose false:  False
ellipsoid norm of secret after posteriori_integration:  0.8803137381115889
       Attack Estimation      
 dim=704 	 δ=1.007146 	 β=161.51  
  
########beta after only integrating posteriori distribution######## 161.51177477315065
expect false:  False
ellipsoid norm of secret after posteriori_integration:  0.8803137381115889
ellipsoid norm of secret after posteriori_integration:  0.8803137375850056
ellipsoid norm of secret after posteriori_integration:  0.8803137382163619
to check here is scaled
VOL OF ELLIP after Posteriori Phase:  -infinity
1 ebdd scaled to <= 1? False
----------------------
Bbbbbb@hints:  [0.8

/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:163: RuntimeWarning: invalid value encountered in log


(Solution) pi = 0.556474355775639
alpha = 0.634004039374759
VOL OF ELLIP_OUTPUT:  -156.013232693071
ellip_norm: 
0.8447252742893454
----------------var of guess coordinates----------------
       Attack Estimation      
 dim=704 	 δ=1.007204 	 β=159.31  
  
2 ebdd scaled to <= 1? False
############beta after integrating SCA info########### 159.3113862003423
expect false:  False
ellipsoid norm of secret after hints_integration:  0.8447252742893454
to check here is scaled
VOL OF ELLIP at Intersect Phase:  -infinity
scaled? expect <= dim here False
      Build chopped EBDD from previous LWE      
----------------------
Sigma_unscaled@reinitial:  [0.0000484000000000000     0.000000000000000   2.00000000000000e-7 0.0000107000000000000  -8.40000000000000e-6]
[    0.000000000000000     0.236817800000000     0.103954700000000   -0.0906568000000000   -0.0112165000000000]
[  2.00000000000000e-7     0.103954700000000      3.42936060000000     0.542134600000000     0.564954900000000]
[0.0000107000

In [20]:
# def main(params, nb_tests_per_params, thres_prob):
"""  Averaging
The following averages the measures to get accurate data
for the paper. The averaging mode is quite long.
"""
params = 'CCS2'

eps = 10**-10
n, m, q, frodo_distribution, D_s, av_prev, var_prev, sca_seeds, param = set_parameters(params)

nb_tests_per_params = 1
thres_prob = 0.995

A_list = [0]*nb_tests_per_params
b_list = [0]*nb_tests_per_params
e_vec_list = [0]*nb_tests_per_params
ebdd_list = [0]*nb_tests_per_params
B_Sigma_list = [0]*nb_tests_per_params
measured_list = [0]*nb_tests_per_params


A2_list = [0]*nb_tests_per_params
b2_list = [0]*nb_tests_per_params
e_vec2_list = [0]*nb_tests_per_params
ebdd2_list = [0]*nb_tests_per_params
B_Sigma2_list = [0]*nb_tests_per_params
measured2_list = [0]*nb_tests_per_params

for ind in range(nb_tests_per_params):
    print("Sample ", ind)
    ####################Initial Phase#######################
    A, b, e_vec, ebdd, B_Sigma = initialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo,
                                                                   n,
                                                                   q, m, D_s, D_s, homogeneous=False, verbosity=1)
    
    A_list[ind] = matrix(A)
    b_list[ind] = matrix(b)
    e_vec_list[ind] = matrix(e_vec)
    
    B1_temp = matrix(ebdd.B)
    S1_temp = matrix(ebdd.S)
    mu1_temp = matrix(ebdd.mu)
    u1_temp = matrix(ebdd.u)
    ebdd_list[ind] = EBDD_non_homo(B1_temp, S1_temp, mu1_temp, u1_temp)
    
    B_Sigma_list[ind] = B_Sigma
    
    
    ###
    A2 = matrix(A)
    b2 = matrix(b)
    e_vec2 = matrix(e_vec)
    s2 = matrix(ebdd.u[:, :n])
    _, _, _, ebdd2, B_Sigma2 = initialize_kannan_ellipsoid_from_LWE_derandom(EBDD_non_homo,
                                                                              n, q, m, D_s, D_s, 
                                                                              homogeneous=False, verbosity=1,
                                                                              A = A2, s = s2, e_vec = e_vec2)
    
    
    A2_list[ind] = A2
    b2_list[ind] = b2
    e_vec2_list[ind] = e_vec2
    
    B2_temp = matrix(ebdd2.B)
    S2_temp = matrix(ebdd2.S)
    mu2_temp = matrix(ebdd2.mu)
    u2_temp = matrix(ebdd2.u)
    ebdd2_list[ind] = EBDD_non_homo(B2_temp, S2_temp, mu2_temp, u2_temp)
    
#     ebdd2_list[ind] = EBDD_non_homo(ebdd2)
    B_Sigma2_list[ind] = B_Sigma2
    B_Sigma3 = matrix(B_Sigma)
    print(B_Sigma3 == B_Sigma2)
    
    
    measures = data_to_measure(sca_seeds, params, param, frodo_distribution)
    measured = [simu_measured(ebdd.u[0, i], measures) for i in range(0, n)]
    
    measured_list[ind] = measured
                                                                          
    ###                                                                      
    measured2 = list(measured)
    ###    
    measured2_list[ind] = measured2

 Set of parameters: CCS2 
Sample  0
      Build EBDD from LWE      
 n=592 	 m=592 	 q=4096 
ellipnorm of secret@initialize:  0.985961571327425
      Build EBDD from LWE      
 n=592 	 m=592 	 q=4096 
ellipnorm of secret@initialize_derandom:  0.985961571327425
True


In [21]:
#guess_then_intersect
for ind in range(nb_tests_per_params):
    A = A_list[ind]
    print(A.ncols())
    b = b_list[ind]
    e_vec = e_vec_list[ind]
    ebdd = ebdd_list[ind]
    B_Sigma = B_Sigma_list[ind]
    measured = measured_list[ind]
#     beta_origin, beta_post, beta_guess, beta_intersect, beta_short, dvol_origin, dvol_post, dvol_guess, dvol_intersect, dvol_short, p_success, guesses, list_cov_prev, list_cov_curr = guess_then_intersect(A, b, e_vec, ebdd, B_Sigma, measured)
    A1_guess, b1_guess, e1_vec, q, short_mu1_s, short_mu1_e, ebdd1_guess = guess_then_intersect(A, b, e_vec, ebdd, B_Sigma, measured)
    
#intersect_then_guess
for ind in range(nb_tests_per_params):
    A2 = A2_list[ind]
    b2 = b2_list[ind]
    e_vec2 = e_vec2_list[ind]
    ebdd2 = ebdd2_list[ind]
    B_Sigma2 = B_Sigma2_list[ind]
    measured2 = measured2_list[ind]
#     beta_origin, beta_post, beta_guess, beta_intersect, beta_short, dvol_origin, dvol_post, dvol_guess, dvol_intersect, dvol_short, p_success, guesses, list_cov_prev, list_cov_curr = intersect_then_guess(A2, b2, e_vec2, ebdd2, B_Sigma2, measured2)
    A2_guess, b2_guess, e2_vec, q, short_mu2_s, short_mu2_e, ebdd2_guess = intersect_then_guess(A2, b2, e_vec2, ebdd2, B_Sigma2, measured2)    
    

592
1 scaled to <= 1? False
       Attack Estimation      
 dim=1184 	 δ=1.003678 	 β=448.31  
  
########initial beta########:  448.3061138772308
1.5 scaled to <= 1? False
to check here is scaled
VOL OF ELLIP at Initial Phase:  -9755.95758289843
scaled@funct posteriori_integration?:  False
None in mean_e@mean_s_c_update
scaled or not suppose false:  False
ellipsoid norm of secret after posteriori_integration:  0.6517112318584566
expect false:  False
ellipsoid norm of secret after posteriori_integration:  0.6517112318584566
ellipsoid norm of secret after posteriori_integration:  0.6517112319188811
       Attack Estimation      
 dim=1184 	 δ=1.005618 	 β=238.95  
  
########beta after only integrating posteriori distribution######## 238.9502464481502
to check here is scaled
VOL OF ELLIP after Posteriori Phase:  -infinity
scaled? expect <= dim here False
      Build chopped EBDD from previous LWE      
----------------------
Sigma_unscaled@reinitial:  [0.0000101000000000000     0.000000

84 VOL OF ELLIP_LWE:  -infinity 225 225
85 VOL OF ELLIP_LWE:  -infinity 225 225
86 VOL OF ELLIP_LWE:  -infinity 225 225
87 VOL OF ELLIP_LWE:  -infinity 225 225
88 VOL OF ELLIP_LWE:  -infinity 225 225
89 VOL OF ELLIP_LWE:  -infinity 225 225
92 VOL OF ELLIP_LWE:  -infinity 225 225
93 VOL OF ELLIP_LWE:  -infinity 225 225
94 VOL OF ELLIP_LWE:  -infinity 225 225
95 VOL OF ELLIP_LWE:  -infinity 225 225
96 VOL OF ELLIP_LWE:  -infinity 225 225
97 VOL OF ELLIP_LWE:  -infinity 225 225
98 VOL OF ELLIP_LWE:  -infinity 225 225
99 VOL OF ELLIP_LWE:  -infinity 225 225
100 VOL OF ELLIP_LWE:  -infinity 225 225
101 VOL OF ELLIP_LWE:  -infinity 225 225
102 VOL OF ELLIP_LWE:  -infinity 225 225
103 VOL OF ELLIP_LWE:  -infinity 225 225
105 VOL OF ELLIP_LWE:  -infinity 225 225
106 VOL OF ELLIP_LWE:  -infinity 225 225
108 VOL OF ELLIP_LWE:  -infinity 225 225
109 VOL OF ELLIP_LWE:  -infinity 225 225
110 VOL OF ELLIP_LWE:  -infinity 225 225
111 VOL OF ELLIP_LWE:  -infinity 225 225
113 VOL OF ELLIP_LWE:  -infini

/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:543: RuntimeWarning: invalid value encountered in log


(Solution) pi = 0.690896916769295
alpha = 0.199102486793919
X*X_inv MSE = 6.817316464872943e-36
VOL OF ELLIP_OUTPUT:  458.160214352615
dimension match check @hints 225 225 225
dimension match check @hints 592 592 592
None in mean_e@mean_s_c_update
ellip_norm: 
1.4607694223340246
       Attack Estimation      
 dim=817 	 δ=1.007562 	 β=146.42  
  
2 ebdd_guess scaled to <= 1? False
############beta after integrating SCA info########### 146.4245479378616
expect false:  False
ellipsoid norm of secret after hints_integration:  1.0079802517288243
to check here is scaled
VOL OF ELLIP at Intersect Phase:  -infinity
      intersect_then_guess      
1 scaled to <= 1? False
       Attack Estimation      
 dim=1184 	 δ=1.003678 	 β=448.31  
  
########initial beta########:  448.3061138772308
1.5 scaled to <= 1? False
to check here is scaled
VOL OF ELLIP at Initial Phase:  -9755.95758289843
scaled@funct posteriori_integration?:  False
None in mean_e@mean_s_c_update
scaled or not suppose false:  Fa

/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:163: RuntimeWarning: invalid value encountered in log


(Solution) pi = 0.623562435731394
alpha = 0.597291776444763
VOL OF ELLIP_OUTPUT:  -2438.19555290811
ellip_norm: 
0.6542837576765379
----------------var of guess coordinates----------------
var is greater after intersect:  26 0.005424884203866257 0.008536678354930443
var is greater after intersect:  42 0.0028707126603523094 0.004534998450691462
var is greater after intersect:  72 0.0001529731191585634 0.00024266487437118875
var is greater after intersect:  106 0.0006272056007838056 0.0009942287721180084
var is greater after intersect:  111 0.00018979292025183924 0.0003010560139239558
var is greater after intersect:  135 0.0035734337584854042 0.005639074300503201
var is greater after intersect:  163 0.00015522827482469622 0.00024624143039005786
var is greater after intersect:  174 0.0011737183726189648 0.001858990281488118
var is greater after intersect:  190 0.003435679740095815 0.005422829317779534
var is greater after intersect:  231 0.00017823681943154851 0.00028273033693338714
var i

In [304]:
load("../framework/instance_gen.sage")
load("../framework/geometry_twist.sage")
load("../framework/geometry.sage")
demo = False
from scipy.io import loadmat
import random

In [287]:
##################2Short Vector Phase#################
n2_short = A2_guess.ncols()
m2_short = A2_guess.nrows()
short_mu2_s = ebdd2_guess.mu[:, :n2_short]
short_mu2_e = vec([0] * m2_short)

dbdd2_short = switch_to_kannan_embedding(DBDD_predict_diag, A2_guess, b2_guess, e2_vec, q, short_mu2_s, short_mu2_e, ebdd2_guess)
print("check check")
dbdd2_short_check = switch_to_kannan_embedding(DBDD, A2_guess, b2_guess, e2_vec, q, short_mu2_s, short_mu2_e, ebdd2_guess)

n2_short = A2_guess.ncols()
m2_short = A2_guess.nrows()
dim2 = n2_short + m2_short
print("ellipsoid norm of secret after switching to Kannan: ", scal((dbdd2_short_check.u - dbdd2_short_check.mu)[:,:dim2 - 1] * matrix(np_inv(dbdd2_short_check.S[:dim2 - 1, :dim2 - 1]))/(dim2) * (((dbdd2_short_check.u - dbdd2_short_check.mu)[:,:dim2 - 1])).T))

#       scal((dbdd2_short_check.u - dbdd2_short_check.mu) * matrix(np_inv(dbdd2_short_check.S))/(n + m) * ((dbdd2_short_check.u - dbdd2_short_check.mu)).T))

print("is q vector in ellipsoid? ", )
q_s = concatenate([[q], [0] * (n2_short + m2_short)])
q_e = concatenate([[0] * n2_short, [q], [0] * m2_short])
print("q in vec_s", scal((q_s - dbdd2_short_check.mu)[:,:dim2 - 1] * matrix(np_inv(dbdd2_short_check.S[:dim2 - 1, :dim2 - 1]))/(dim2) * (((q_s - dbdd2_short_check.mu))[:,:dim2 - 1]).T))
print("q in vec_e", scal((q_e - dbdd2_short_check.mu)[:,:dim2 - 1] * matrix(np_inv(dbdd2_short_check.S[:dim2 - 1, :dim2 - 1]))/(dim2) * (((q_e - dbdd2_short_check.mu)[:,:dim2 - 1])).T))

dbdd2_short.integrate_q_vectors(q)#indices=range(n, n + m) 
(beta2_short, _) = dbdd2_short.estimate_attack()
dvol2_short = dbdd2_short.dvol
print("########beta after intersect + guess########", beta2_short)
#     print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
print("ellip_norm after short vector phase: ")
# print(scal((dbdd_short.u - dbdd_short.mu) * matrix(np_inv(dbdd_short.S))/(dbdd_short.u.ncols()) * ((dbdd_short.u - dbdd_short.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((dbdd_short.u - dbdd_short.mu) * dbdd_short.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
# print("VOL OF ELLIP at Short Vector Phase: ", ln_vol_ellip(dbdd_short))


n, m 181 352
181
B dim: 534 534
D dim: 534 534
suppose false@switch_to_kannan_embedding:  False
----------------------
S_s_e_p@switch:  [0.0485411000000000  0.000000000000000  0.000000000000000  0.000000000000000  0.000000000000000]
[ 0.000000000000000  0.162837300000000  0.000000000000000  0.000000000000000  0.000000000000000]
[ 0.000000000000000  0.000000000000000   1.31250000000000  0.000000000000000  0.000000000000000]
[ 0.000000000000000  0.000000000000000  0.000000000000000   1.31250000000000  0.000000000000000]
[ 0.000000000000000  0.000000000000000  0.000000000000000  0.000000000000000   1.31250000000000] 533
S_s_e dim: 533 533
S dim: 534 534
vec_s dim: 1 181
352
181
mu dim: 1 534
u dim: 1 534
check check
n, m 181 352
181
B dim: 534 534
D dim: 534 534
suppose false@switch_to_kannan_embedding:  False
----------------------
S_s_e_p@switch:  [0.0485411000000000  0.000000000000000  0.000000000000000  0.000000000000000  0.000000000000000]
[ 0.000000000000000  0.162837300000000  0.00

integrate_q_vectors 429 0
 integrate short vector hint   	 Worthy hint !   dim=428, δ=1.00984815, β=88.02 
integrate_q_vectors 428 0
 integrate short vector hint   	 Worthy hint !   dim=427, δ=1.00984412, β=87.89 
integrate_q_vectors 427 0
 integrate short vector hint   	 Worthy hint !   dim=426, δ=1.00985187, β=87.77 
integrate_q_vectors 426 0
 integrate short vector hint   	 Worthy hint !   dim=425, δ=1.00985958, β=87.65 
integrate_q_vectors 425 0
 integrate short vector hint   	 Worthy hint !   dim=424, δ=1.00986726, β=87.53 
integrate_q_vectors 424 0
 integrate short vector hint   	 Worthy hint !   dim=423, δ=1.00987490, β=87.41 
integrate_q_vectors 423 0
 integrate short vector hint   	 Worthy hint !   dim=422, δ=1.00988250, β=87.29 
integrate_q_vectors 422 0
 integrate short vector hint   	 Worthy hint !   dim=421, δ=1.00989006, β=87.17 
integrate_q_vectors 421 0
 integrate short vector hint   	 Worthy hint !   dim=420, δ=1.00989758, β=87.05 
integrate_q_vectors 420 0
 integrate 

In [302]:
##################Short Vector Phase after Initial#################
A_temp, b_temp, e_temp_vec, ebdd_temp, B_Sigma_temp = initialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo,
                                                                   n,
                                                                   q, m, D_s, D_s, homogeneous=False, verbosity=1)
    

A_k, b_k, dbdd_k = initialize_from_LWE_instance(DBDD_predict_diag,
                                               n,
                                               q, m, D_s, D_s, verbosity=1)
n_short = A_temp.ncols()
m_short = A_temp.nrows()
short_mu_s = ebdd_temp.mu[:, :n_short]
short_mu_e = vec([0] * m_short)
(beta_temp, _) =  ebdd_temp.estimate_attack()
print("beta at initial: ",beta_temp)
dbdd_short22 = switch_to_kannan_embedding(DBDD_predict_diag, A_temp, b_temp, e_temp_vec, q, short_mu_s, short_mu_e, ebdd_temp)
# print("check check")
# dbdd2_short_check = switch_to_kannan_embedding(DBDD, A2_guess, b2_guess, e2_vec, q, short_mu2_s, short_mu2_e, ebdd2_guess)

# n2_short = A2_guess.ncols()
# m2_short = A2_guess.nrows()
# dim2 = n2_short + m2_short
# print("ellipsoid norm of secret after switching to Kannan: ", scal((dbdd2_short_check.u - dbdd2_short_check.mu)[:,:dim2 - 1] * matrix(np_inv(dbdd2_short_check.S[:dim2 - 1, :dim2 - 1]))/(dim2) * (((dbdd2_short_check.u - dbdd2_short_check.mu)[:,:dim2 - 1])).T))

# #       scal((dbdd2_short_check.u - dbdd2_short_check.mu) * matrix(np_inv(dbdd2_short_check.S))/(n + m) * ((dbdd2_short_check.u - dbdd2_short_check.mu)).T))

# print("is q vector in ellipsoid? ", )
# q_s = concatenate([[q], [0] * (n2_short + m2_short)])
# q_e = concatenate([[0] * n2_short, [q], [0] * m2_short])
# print("q in vec_s", scal((q_s - dbdd2_short_check.mu)[:,:dim2 - 1] * matrix(np_inv(dbdd2_short_check.S[:dim2 - 1, :dim2 - 1]))/(dim2) * (((q_s - dbdd2_short_check.mu))[:,:dim2 - 1]).T))
# print("q in vec_e", scal((q_e - dbdd2_short_check.mu)[:,:dim2 - 1] * matrix(np_inv(dbdd2_short_check.S[:dim2 - 1, :dim2 - 1]))/(dim2) * (((q_e - dbdd2_short_check.mu)[:,:dim2 - 1])).T))

dbdd_short22.integrate_q_vectors(q, indices=range(n, n + m))#indices=range(n, n + m) 
dbdd_k.integrate_q_vectors(q, indices=range(n, n + m))#indices=range(n, n + m) 

(beta_short22, _) = dbdd_short22.estimate_attack()
dvol_short22 = dbdd_short22.dvol
print("beta after short: ", beta_short22)
# print("########beta after intersect + guess########", beta_short)
#     print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
print("ellip_norm after short vector phase: ")
# print(scal((dbdd_short.u - dbdd_short.mu) * matrix(np_inv(dbdd_short.S))/(dbdd_short.u.ncols()) * ((dbdd_short.u - dbdd_short.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((dbdd_short.u - dbdd_short.mu) * dbdd_short.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
# print("VOL OF ELLIP at Short Vector Phase: ", ln_vol_ellip(dbdd_short))


      Build EBDD from LWE      
 n=352 	 m=352 	 q=2048 
ellipnorm of secret@initialize:  1.03030303030303
      Build DBDD from LWE      
 n=352 	 m=352 	 q=2048 
       Attack Estimation      
 dim=704 	 δ=1.005205 	 β=269.07  
  
beta at initial:  269.0712906739275
n, m 352 352
352
B dim: 705 705
D dim: 705 705
suppose false@switch_to_kannan_embedding:  False
----------------------
S_s_e_p@switch:  [ 1.31250000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000  1.31250000000000 0.000000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000  1.31250000000000 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000 0.000000000000000  1.31250000000000 0.000000000000000]
[0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000  1.31250000000000] 704
----------------------
S_s_e@switch:  [ 1.31250000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000]
[

In [309]:

n = 352
m = 352
q = 2 ** 11
frodo_distribution = [22528, 15616, 5120, 768]
D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
sca_seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param = 0


In [307]:
load("../framework/instance_gen.sage")
demo = False
from scipy.io import loadmat
import random

In [311]:
A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                           n,
                                           q, m, D_s, D_s, verbosity=1)
dbdd.integrate_q_vectors(q, indices=range(n, n + m))

      Build DBDD from LWE      
 n=352 	 m=352 	 q=2048 
       Integrating q-vectors      
integrate_q_vectors 705 0
 integrate short vector hint   	 Unworthy hint, Rejected. 


[]

In [232]:
load("../framework/instance_gen.sage")
load("../framework/geometry_twist.sage")
load("../framework/geometry.sage")
demo = False
from scipy.io import loadmat
import random

In [189]:
n_short = A_guess.ncols()
m_short = A.nrows()
short_mu_s = ebdd.mu[:, :n_short]
short_mu_e = vec([0] * m_short)

In [326]:
load("exploiting_SCA_from_Bos_et_al.sage")

 --- Demonstration mode (no redundancy of the attacks) --- 
 Set of parameters: CCS1 
 Attack without hints:  268.83 bikz 
      Build DBDD from LWE      
 n=352 	 m=352 	 q=2048 
 [...50]   integrate approx hint   (aposteriori)   u0 = 1.6032376214878201 + χ(σ²=0.241)   	 Worthy hint !   dim=705, δ=1.00520791, β=268.71 
 [...50]   integrate approx hint   (aposteriori)   u50 = 1.0565993812414014e-07 + χ(σ²=0.000)   	 Worthy hint !   dim=705, δ=1.00553274, β=244.76 
 [...50]   integrate approx hint   (aposteriori)   u100 = 1.032887862246761 + χ(σ²=0.038)   	 Worthy hint !   dim=705, δ=1.00582391, β=225.83 
 [...50]   integrate approx hint   (aposteriori)   u150 = 0.002121459732789845 + χ(σ²=0.004)   	 Worthy hint !   dim=705, δ=1.00608834, β=210.48 
 [...50]   integrate approx hint   (aposteriori)   u200 = -1.0000679022290484 + χ(σ²=0.000)   	 Worthy hint !   dim=705, δ=1.00638050, β=195.18 
 [...50]   integrate approx hint   (aposteriori)   u250 = -1.7890200799391351 + χ(σ²=0.699)   	 W

integrate_q_vectors 452 353
integrate_q_vectors 451 353
integrate_q_vectors 450 353
integrate_q_vectors 449 353
integrate_q_vectors 448 353
integrate_q_vectors 447 353
integrate_q_vectors 446 353
integrate_q_vectors 445 353
integrate_q_vectors 444 353
integrate_q_vectors 443 353
integrate_q_vectors 442 353
integrate_q_vectors 441 353
integrate_q_vectors 440 353
integrate_q_vectors 439 353
integrate_q_vectors 438 353
integrate_q_vectors 437 353
integrate_q_vectors 436 353
integrate_q_vectors 435 353
integrate_q_vectors 434 353
integrate_q_vectors 433 353
integrate_q_vectors 432 353
integrate_q_vectors 431 353
integrate_q_vectors 430 353
 dim=430 	 delta=1.008309 	 beta=123.53 	 guesses=  46   Proba success = 0.999994781973638 
 [...50]  integrate_q_vectors 380 353
 dim=380 	 delta=1.008750 	 beta=111.91 	 guesses=  96   Proba success = 0.999636573445876 
 [...50]   dim=330 	 delta=1.009715 	 beta=90.56 	 guesses= 146   Proba success = 0.991405703379831 
 [...50]   dim=280 	 delta=1.0116

integrate_q_vectors 1040 593
integrate_q_vectors 1039 593
integrate_q_vectors 1038 593
integrate_q_vectors 1037 593
integrate_q_vectors 1036 593
integrate_q_vectors 1035 593
integrate_q_vectors 1034 593
integrate_q_vectors 1033 593
integrate_q_vectors 1032 593
integrate_q_vectors 1031 593
integrate_q_vectors 1030 593
integrate_q_vectors 1029 593
integrate_q_vectors 1028 593
integrate_q_vectors 1027 593
integrate_q_vectors 1026 593
integrate_q_vectors 1025 593
integrate_q_vectors 1024 593
integrate_q_vectors 1023 593
integrate_q_vectors 1022 593
integrate_q_vectors 1021 593
integrate_q_vectors 1020 593
integrate_q_vectors 1019 593
integrate_q_vectors 1018 593
integrate_q_vectors 1017 593
integrate_q_vectors 1016 593
integrate_q_vectors 1015 593
integrate_q_vectors 1014 593
integrate_q_vectors 1013 593
integrate_q_vectors 1012 593
integrate_q_vectors 1011 593
integrate_q_vectors 1010 593
integrate_q_vectors 1009 593
integrate_q_vectors 1008 593
integrate_q_vectors 1007 593
integrate_q_ve

integrate_q_vectors 747 593
integrate_q_vectors 746 593
integrate_q_vectors 745 593
integrate_q_vectors 744 593
integrate_q_vectors 743 593
integrate_q_vectors 742 593
integrate_q_vectors 741 593
integrate_q_vectors 740 593
integrate_q_vectors 739 593
integrate_q_vectors 738 593
integrate_q_vectors 737 593
integrate_q_vectors 736 593
integrate_q_vectors 735 593
integrate_q_vectors 734 593
integrate_q_vectors 733 593
integrate_q_vectors 732 593
integrate_q_vectors 731 593
integrate_q_vectors 730 593
integrate_q_vectors 729 593
integrate_q_vectors 728 593
integrate_q_vectors 727 593
integrate_q_vectors 726 593
integrate_q_vectors 725 593
integrate_q_vectors 724 593
integrate_q_vectors 723 593
integrate_q_vectors 722 593
integrate_q_vectors 721 593
integrate_q_vectors 720 593
integrate_q_vectors 719 593
integrate_q_vectors 718 593
integrate_q_vectors 717 593
integrate_q_vectors 716 593
integrate_q_vectors 715 593
integrate_q_vectors 714 593
integrate_q_vectors 713 593
integrate_q_vectors 

integrate_q_vectors 1252 753
integrate_q_vectors 1251 753
integrate_q_vectors 1250 753
integrate_q_vectors 1249 753
integrate_q_vectors 1248 753
integrate_q_vectors 1247 753
integrate_q_vectors 1246 753
integrate_q_vectors 1245 753
integrate_q_vectors 1244 753
integrate_q_vectors 1243 753
integrate_q_vectors 1242 753
integrate_q_vectors 1241 753
integrate_q_vectors 1240 753
integrate_q_vectors 1239 753
integrate_q_vectors 1238 753
integrate_q_vectors 1237 753
integrate_q_vectors 1236 753
integrate_q_vectors 1235 753
integrate_q_vectors 1234 753
integrate_q_vectors 1233 753
integrate_q_vectors 1232 753
integrate_q_vectors 1231 753
integrate_q_vectors 1230 753
integrate_q_vectors 1229 753
integrate_q_vectors 1228 753
integrate_q_vectors 1227 753
integrate_q_vectors 1226 753
integrate_q_vectors 1225 753
integrate_q_vectors 1224 753
integrate_q_vectors 1223 753
integrate_q_vectors 1222 753
integrate_q_vectors 1221 753
integrate_q_vectors 1220 753
integrate_q_vectors 1219 753
integrate_q_ve

...

integrate_q_vectors 1181 641
integrate_q_vectors 1180 641
integrate_q_vectors 1179 641
integrate_q_vectors 1178 641
integrate_q_vectors 1177 641
integrate_q_vectors 1176 641
integrate_q_vectors 1175 641
integrate_q_vectors 1174 641
integrate_q_vectors 1173 641
integrate_q_vectors 1172 641
integrate_q_vectors 1171 641
integrate_q_vectors 1170 641
integrate_q_vectors 1169 641
integrate_q_vectors 1168 641
integrate_q_vectors 1167 641
integrate_q_vectors 1166 641
integrate_q_vectors 1165 641
integrate_q_vectors 1164 641
integrate_q_vectors 1163 641
integrate_q_vectors 1162 641
integrate_q_vectors 1161 641
integrate_q_vectors 1160 641
integrate_q_vectors 1159 641
integrate_q_vectors 1158 641
integrate_q_vectors 1157 641
integrate_q_vectors 1156 641
integrate_q_vectors 1155 641
integrate_q_vectors 1154 641
integrate_q_vectors 1153 641
integrate_q_vectors 1152 641
integrate_q_vectors 1151 641
integrate_q_vectors 1150 641
integrate_q_vectors 1149 641
integrate_q_vectors 1148 641
integrate_q_ve

integrate_q_vectors 670 641
integrate_q_vectors 669 641
integrate_q_vectors 668 641
integrate_q_vectors 667 641
integrate_q_vectors 666 641
integrate_q_vectors 665 641
 dim=665 	 delta=1.006909 	 beta=170.95 	 guesses= 282   Proba success = 0.0000733035317763628 
 [...50]   dim=615 	 delta=1.007890 	 beta=135.73 	 guesses= 332   Proba success = 5.59642167679555e-9 
 [...50]   dim=565 	 delta=1.009138 	 beta=102.75 	 guesses= 382   Proba success = 1.59089262970889e-14 
 [...50]   dim=515 	 delta=1.010742 	 beta=71.68 	 guesses= 432   Proba success = 2.18419468591247e-21 
 [...50]   dim=465 	 delta=1.012728 	 beta=38.45 	 guesses= 482   Proba success = 4.52123708719787e-30 
 Set of parameters: NIST2 
 Attack without hints:  708.06 bikz 
      Build DBDD from LWE      
 n=976 	 m=976 	 q=65536 
 [...50]   integrate approx hint   (aposteriori)   u0 = 3.0081333855020733 + χ(σ²=0.050)   	 Worthy hint !   dim=1953, δ=1.00264569, β=707.86 
 [...50]   integrate approx hint   (aposteriori)   u50

integrate_q_vectors 1848 977
integrate_q_vectors 1847 977
integrate_q_vectors 1846 977
integrate_q_vectors 1845 977
integrate_q_vectors 1844 977
integrate_q_vectors 1843 977
integrate_q_vectors 1842 977
integrate_q_vectors 1841 977
integrate_q_vectors 1840 977
integrate_q_vectors 1839 977
integrate_q_vectors 1838 977
integrate_q_vectors 1837 977
integrate_q_vectors 1836 977
integrate_q_vectors 1835 977
integrate_q_vectors 1834 977
integrate_q_vectors 1833 977
integrate_q_vectors 1832 977
integrate_q_vectors 1831 977
integrate_q_vectors 1830 977
integrate_q_vectors 1829 977
integrate_q_vectors 1828 977
integrate_q_vectors 1827 977
integrate_q_vectors 1826 977
integrate_q_vectors 1825 977
integrate_q_vectors 1824 977
integrate_q_vectors 1823 977
integrate_q_vectors 1822 977
integrate_q_vectors 1821 977
integrate_q_vectors 1820 977
integrate_q_vectors 1819 977
integrate_q_vectors 1818 977
integrate_q_vectors 1817 977
integrate_q_vectors 1816 977
integrate_q_vectors 1815 977
integrate_q_ve

integrate_q_vectors 1565 977
integrate_q_vectors 1564 977
integrate_q_vectors 1563 977
integrate_q_vectors 1562 977
integrate_q_vectors 1561 977
integrate_q_vectors 1560 977
integrate_q_vectors 1559 977
integrate_q_vectors 1558 977
integrate_q_vectors 1557 977
integrate_q_vectors 1556 977
integrate_q_vectors 1555 977
integrate_q_vectors 1554 977
integrate_q_vectors 1553 977
integrate_q_vectors 1552 977
integrate_q_vectors 1551 977
integrate_q_vectors 1550 977
integrate_q_vectors 1549 977
integrate_q_vectors 1548 977
integrate_q_vectors 1547 977
integrate_q_vectors 1546 977
integrate_q_vectors 1545 977
integrate_q_vectors 1544 977
integrate_q_vectors 1543 977
integrate_q_vectors 1542 977
integrate_q_vectors 1541 977
integrate_q_vectors 1540 977
integrate_q_vectors 1539 977
integrate_q_vectors 1538 977
integrate_q_vectors 1537 977
integrate_q_vectors 1536 977
integrate_q_vectors 1535 977
integrate_q_vectors 1534 977
integrate_q_vectors 1533 977
integrate_q_vectors 1532 977
integrate_q_ve

 [...50]  integrate_q_vectors 1013 977
integrate_q_vectors 1012 977
integrate_q_vectors 1011 977
integrate_q_vectors 1010 977
integrate_q_vectors 1009 977
integrate_q_vectors 1008 977
integrate_q_vectors 1007 977
integrate_q_vectors 1006 977
integrate_q_vectors 1005 977
integrate_q_vectors 1004 977
integrate_q_vectors 1003 977
integrate_q_vectors 1002 977
integrate_q_vectors 1001 977
integrate_q_vectors 1000 977
integrate_q_vectors 999 977
integrate_q_vectors 998 977
integrate_q_vectors 997 977
integrate_q_vectors 996 977
integrate_q_vectors 995 977
integrate_q_vectors 994 977
integrate_q_vectors 993 977
integrate_q_vectors 992 977
integrate_q_vectors 991 977
integrate_q_vectors 990 977
integrate_q_vectors 989 977
integrate_q_vectors 988 977
integrate_q_vectors 987 977
integrate_q_vectors 986 977
integrate_q_vectors 985 977
integrate_q_vectors 984 977
integrate_q_vectors 983 977
integrate_q_vectors 982 977
 dim=982 	 delta=1.005229 	 beta=267.12 	 guesses= 322   Proba success = 0.25100

In [12]:
def ln_vol_ellip(ebdd):
    if not ebdd.scaled == False:
        dim = ebdd.S.ncols()
        return ln((ebdd.S * dim).det()).n()
    else:
        print("to check here is scaled")
        return ln((ebdd.S).det()).n()
        

In [10]:
def guess_then_intersect(A, b, e_vec, ebdd, B_Sigma, measured, factor = 0):
    ####################Initial Phase#######################
    print("1 scaled to <= 1?", ebdd.scaled)
    (beta_origin, _) = ebdd.estimate_attack()
    dvol_origin =  ebdd.dvol
    print("########initial beta########: ", beta_origin)
    print("1.5 scaled to <= 1?", ebdd.scaled)
    print("VOL OF ELLIP at Initial Phase: ", ln_vol_ellip(ebdd))

    ####################Posteriori Phase#######################                                                           
    list_av, list_var, intersect_indices, p_success, guesses, guess_indices, _ = count_SCA_thres(var_prev, measured, thres_prob)

    mu_post, B_post = posteriori_integration_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, n, m, factor)
    print("expect false: ", ebdd.scaled)
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - ebdd.mu)).T))
#     print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * ebdd.S.inverse()/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * B_post * B_post.T/(n + m) * ((ebdd.u - ebdd.mu)).T))
    
    (beta_post, _) = ebdd.estimate_attack()
    dvol_post = ebdd.dvol
    print("########beta after only integrating posteriori distribution########", beta_post)
    print("VOL OF ELLIP after Posteriori Phase: ", ln_vol_ellip(ebdd))
    ####################---Guess Phase---###############
    new_mu_s = ebdd.mu[:, :n]
    new_mu_e = vec([0] * m)
    print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    Sigma_unscaled = ebdd.S  #non-homo, scale to <= dim 
    A_guess, b_guess, e_vec, ebdd_guess, B_Sigma_guess = reinitialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo, A, ebdd.u, e_vec, q, guess_indices, 
												new_mu_s, 
												new_mu_e, 
												Sigma_unscaled,
												homogeneous=False)
    n_guess = A_guess.ncols()
    m_guess = A_guess.nrows()
    (beta_guess, _) = ebdd_guess.estimate_attack()
    dvol_guess = ebdd_guess.dvol
    print("########beta after post + guess########", beta_guess)
    print("scaled? expect <= dim here", ebdd_guess.scaled) #suppose unscaled
    print("ellip_norm: ")
    print(scal((ebdd_guess.u - ebdd_guess.mu) * matrix(np_inv(ebdd_guess.S))/(n_guess + m_guess) * ((ebdd_guess.u - ebdd_guess.mu)).T))
#     print("ellipsoid norm of secret after post + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * ebdd_guess.S.inverse()/(n_guess + m_guess) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("ellipsoid norm of secret after post + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n_guess + m_guess) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("VOL OF ELLIP at Guess Phase: ", ln_vol_ellip(ebdd_guess))
    ####################Intersect Phase#######################
    print("1 ebdd_guess scaled to <= 1?", ebdd_guess.scaled)
    list_av_guess = value_list_update(guess_indices, list_av)
    list_var_guess = value_list_update(guess_indices, list_var)

    intersect_indices_guess = intersect_indices_update(guess_indices, intersect_indices, n) #here n is the original n
    mu_s_intersect, Sigma_s_intersect, Sigma_intersect = hints_integration_Sigma(ebdd_guess, A_guess, b_guess, B_Sigma_guess, list_av_guess, list_var_guess, intersect_indices_guess, guess_indices, m_guess, n_guess, factor, D_s)
    #     mu_s, Sigma_s, Sigma = hints_integration_prior_twist_full_nosubmatrix(ebdd, B_Sigma, list_av, list_var, intersect_indices, eps, n, m, tangent_indices, thres)
    
    (beta_intersect, _) = ebdd_guess.estimate_attack()
    print("2 ebdd_guess scaled to <= 1?", ebdd_guess.scaled)
    dvol_intersect = ebdd_guess.dvol
    print("############beta after integrating SCA info###########", beta_intersect)
    print("expect false: ", ebdd_guess.scaled)
    print("ellipsoid norm of secret after hints_integration: ", scal((ebdd_guess.u - ebdd_guess.mu) * matrix(np_inv(ebdd_guess.S))/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
#     print("ellipsoid norm of secret after hints_integration: ", scal((ebdd_guess.u - ebdd_guess.mu) * ebdd_guess.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("VOL OF ELLIP at Intersect Phase: ", ln_vol_ellip(ebdd_guess))
    ###################Short Vector Phase#################
    short_mu_s = ebdd.mu[:, :n_guess]
    short_mu_e = vec([0] * m)
    return A_guess, b_guess, e_vec, q, short_mu_s, short_mu_e, ebdd_guess



#     dbdd_short = switch_to_kannan_embedding(DBDD, A_guess, b_guess, e_vec, q, short_mu_s, short_mu_e, ebdd_guess)
#     dbdd_short.integrate_q_vectors(q)#indices=range(n, n + m) 
#     (beta_short, _) = dbdd_short.estimate_attack()
#     dvol_short = dbdd_short.dvol
#     print("########beta after intersect + guess########", beta_short)
# #     print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
#     print("ellip_norm after short vector phase: ")
#     print(scal((dbdd_short.u - dbdd_short.mu) * matrix(np_inv(dbdd_short.S))/(dbdd_short.u.ncols()) * ((dbdd_short.u - dbdd_short.mu)).T))
# #     print("ellipsoid norm of secret after intersect + guess: ", scal((dbdd_short.u - dbdd_short.mu) * dbdd_short.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
# #     print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
#     print("VOL OF ELLIP at Short Vector Phase: ", ln_vol_ellip(dbdd_short))
    
#     return beta_origin, beta_post, beta_guess, beta_intersect, beta_short, dvol_origin, dvol_post, dvol_guess, dvol_intersect, dvol_short, p_success, guesses


In [11]:
def intersect_then_guess(A, b, e_vec, ebdd, B_Sigma, measured, factor = 0):
    logging("     intersect_then_guess     ", style="HEADER")
    ####################Initial Phase#######################
    print("1 scaled to <= 1?", ebdd.scaled)
    (beta_origin, _) = ebdd.estimate_attack()
    dvol_origin =  ebdd.dvol

    print("########initial beta########: ", beta_origin)
    print("1.5 scaled to <= 1?", ebdd.scaled)
    print("VOL OF ELLIP at Initial Phase: ", ln_vol_ellip(ebdd))
    ####################Posteriori Phase#######################
    list_av, list_var, intersect_indices, p_success, guesses, guess_indices, _ = count_SCA_thres(var_prev, measured, thres_prob)

    mu_post, B_post = posteriori_integration_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, n, m, factor)
    (beta_post, _) = ebdd.estimate_attack()
    dvol_post = ebdd.dvol
    print("########beta after only integrating posteriori distribution########", beta_post)
    print("expect false: ", ebdd.scaled)
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * ebdd.S.inverse()/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * B_post * B_post.T/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("VOL OF ELLIP after Posteriori Phase: ", ln_vol_ellip(ebdd))
    ####################Intersect Phase#######################
    
    thres = 0.
    print("1 ebdd scaled to <= 1?", ebdd.scaled)
    Cov_prev = matrix(ebdd.S)
    mu_s_intersect, Sigma_s_intersect, Sigma_intersect = hints_integration_prior_twist_post_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, eps, n, m, factor)
    print("----------------var of guess coordinates----------------")
    Cov_curr = matrix(ebdd.S)
    
    list_cov_prev = []
    list_cov_curr = []
    for i_guess in guess_indices:
#         print(i_guess,list_var[i_guess], Cov_prev[i_guess, i_guess], Cov_curr[i_guess, i_guess])
        if abs(list_var[i_guess] - Cov_prev[i_guess, i_guess]) > 10**(-5):
            print("first two unequal: ", i_guess,list_var[i_guess], Cov_prev[i_guess, i_guess], Cov_curr[i_guess, i_guess])
        if Cov_prev[i_guess, i_guess] < Cov_curr[i_guess, i_guess]:
            print("var is greater after intersect: ", i_guess, Cov_prev[i_guess, i_guess], Cov_curr[i_guess, i_guess])
        
        list_cov_prev += [Cov_prev[i_guess, i_guess]]
        list_cov_curr += [Cov_curr[i_guess, i_guess]]
        
    (beta_intersect, _) = ebdd.estimate_attack()
    print("2 ebdd scaled to <= 1?", ebdd.scaled)
    dvol_intersect = ebdd.dvol

    print("############beta after integrating SCA info###########", beta_intersect)
    print("expect false: ", ebdd.scaled)
    print("ellipsoid norm of secret after hints_integration: ", scal((ebdd.u - ebdd.mu) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - ebdd.mu)).T))
#     print("ellipsoid norm of secret after hints_integration: ", scal((ebdd.u - ebdd.mu) * ebdd.S.inverse()/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("VOL OF ELLIP at Intersect Phase: ", ln_vol_ellip(ebdd))
    ####################---Guess Phase---###############
    new_mu_s = ebdd.mu[:, :n]
    new_mu_e = vec([0] * m)
    print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    Sigma_unscaled = ebdd.S  #non-homo, scale to <= dim 
    A_guess, b_guess, e_vec, ebdd_guess, B_Sigma_guess = reinitialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo, A, ebdd.u, e_vec, q, guess_indices, 
												new_mu_s, 
												new_mu_e, 
												Sigma_unscaled,
												homogeneous=False)
    (beta_guess, _) = ebdd_guess.estimate_attack()
    dvol_guess = ebdd_guess.dvol
    print("########beta after intersect + guess########", beta_guess)
    print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    print("ellip_norm: ")
    print(scal((ebdd_guess.u - ebdd_guess.mu) * matrix(np_inv(ebdd_guess.S))/(ebdd_guess.u.ncols()) * ((ebdd_guess.u - ebdd_guess.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * ebdd_guess.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("VOL OF ELLIP at Guess Phase: ", ln_vol_ellip(ebdd_guess))
    short_mu_s = ebdd.mu[:, :n]
    short_mu_e = vec([0] * m)
    return A_guess, b_guess, e_vec, q, short_mu_s, short_mu_e, ebdd_guess



#     ###################Short Vector Phase#################
#     dbdd_short = switch_to_kannan_embedding(DBDD, A_guess, b_guess, e_vec, q, new_mu_s, new_mu_e, ebdd_guess)
#     dbdd_short.integrate_q_vectors(q)#indices=range(n, n + m) 
#     (beta_short, _) = dbdd_short.estimate_attack()
#     dvol_short = dbdd_short.dvol
#     print("########beta after intersect + guess########", beta_short)
# #     print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
#     print("ellip_norm after short vector phase: ")
#     print(scal((dbdd_short.u - dbdd_short.mu) * matrix(np_inv(dbdd_short.S))/(dbdd_short.u.ncols()) * ((dbdd_short.u - dbdd_short.mu)).T))
# #     print("ellipsoid norm of secret after intersect + guess: ", scal((dbdd_short.u - dbdd_short.mu) * dbdd_short.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
# #     print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
#     print("VOL OF ELLIP at Short Vector Phase: ", ln_vol_ellip(dbdd_short))
    
#     return beta_origin, beta_post, beta_guess, beta_intersect, beta_short, dvol_origin, dvol_post, dvol_guess, dvol_intersect, dvol_short, p_success, guesses, list_cov_prev, list_cov_curr


In [ ]:
##################Short Vector Phase#################
dbdd1_short = switch_to_kannan_embedding(DBDD_predict_diag, A1_guess, b1_guess, e1_vec, q, short_mu1_s, short_mu1_e, ebdd1_guess)
dbdd1_short.integrate_q_vectors(q)#indices=range(n, n + m) 
(beta1_short, _) = dbdd1_short.estimate_attack()
dvol1_short = dbdd1_short.dvol
print("########beta after intersect + guess########", beta1_short)
#     print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
print("ellip_norm after short vector phase: ")
# print(scal((dbdd_short.u - dbdd_short.mu) * matrix(np_inv(dbdd_short.S))/(dbdd_short.u.ncols()) * ((dbdd_short.u - dbdd_short.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((dbdd_short.u - dbdd_short.mu) * dbdd_short.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
# print("VOL OF ELLIP at Short Vector Phase: ", ln_vol_ellip(dbdd_short))


In [131]:
#Testing purpose should be syn with def guess_then_intersect
#guess hint then intersect with short vector at the end
num_total = 0
num_guess_total = 0
proba_success = 0
guesses_total = 0

beta_origin_mean = 0
beta_post_mean = 0
beta_guess_delete_mean = 0
beta_intersect_mean = 0
beta_short_mean = 0

dvol_origin_mean = 0
dvol_post_mean = 0
dvol_guess_delete_mean = 0
dvol_intersect_mean = 0
dvol_short_mean = 0

ebdd_list = [0]*nb_tests_per_params
ebdd_guess_list = [0]*nb_tests_per_params

diff_dvol = 0
dvol_origin_list = [0]*nb_tests_per_params
dvol_post_list = [0]*nb_tests_per_params
dvol_guess_delete_list = [0]*nb_tests_per_params
dvol_intersect_list = [0]*nb_tests_per_params
dvol_short_list = [0]*nb_tests_per_params

A2_list = [0]*nb_tests_per_params
b2_list = [0]*nb_tests_per_params
e_vec2_list = [0]*nb_tests_per_params
ebdd2_list = [0]*nb_tests_per_params
B_Sigma2_list = [0]*nb_tests_per_params
measured2_list = [0]*nb_tests_per_params

thres_prob = 0.995
for ind in range(nb_tests_per_params):
    print("Sample ", ind)
    ####################Initial Phase#######################
    A, b, e_vec, ebdd, B_Sigma = initialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo,
                                                                   n,
                                                                   q, m, D_s, D_s, homogeneous=False, verbosity=0)
    ###
    A2 = matrix(A)
    b2 = matrix(b)
    e_vec2 = matrix(e_vec)
    s2 = matrix(ebdd.u[:, :n])
    _, _, _, ebdd2, B_Sigma2 = initialize_kannan_ellipsoid_from_LWE_derandom(EBDD_non_homo,
                                                                              n, q, m, D_s, D_s, 
                                                                              homogeneous=False, verbosity=0,
                                                                              A = A2, s = s2, e_vec = e_vec2)
    A2_list[ind] = A2
    b2_list[ind] = b2
    e_vec2_list[ind] = e_vec2
    ebdd2_list[ind] = ebdd2
    B_Sigma2_list[ind] = B_Sigma2
    
    B_Sigma2 = matrix(B_Sigma)
    ###                                                             
    
    ####################Initial Phase#######################
    print("1 scaled to <= 1?", ebdd.scaled)
    (beta_origin, _) = ebdd.estimate_attack()
    dvol_origin =  ebdd.dvol
#     dvol_origin_list[ind] = dvol_origin
    print("########initial beta########: ", beta_origin)
    print("1.5 scaled to <= 1?", ebdd.scaled)
    print("VOL OF ELLIP at Initial Phase: ", ln_vol_ellip(ebdd))

    ####################Posteriori Phase#######################                                                           
    list_av, list_var, intersect_indices, p_success, guesses, guess_indices, _ = count_SCA_thres(var_prev, measured, thres_prob)

    mu_post, B_post = posteriori_integration_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, n, m, factor)
    print("expect false: ", ebdd.scaled)
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * ebdd.S.inverse()/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * B_post * B_post.T/(n + m) * ((ebdd.u - ebdd.mu)).T))
    
    (beta_post, _) = ebdd.estimate_attack()
    dvol_post = ebdd.dvol
    print("########beta after only integrating posteriori distribution########", beta_post)
    print("VOL OF ELLIP after Posteriori Phase: ", ln_vol_ellip(ebdd))
    ####################---Guess Phase---###############
    new_mu_s = ebdd.mu[:, :n]
    new_mu_e = vec([0] * m)
    print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    Sigma_unscaled = ebdd.S  #non-homo, scale to <= dim 
    A_guess, b_guess, e_vec, ebdd_guess, B_Sigma_guess = reinitialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo, A, ebdd.u, e_vec, q, guess_indices, 
												new_mu_s, 
												new_mu_e, 
												Sigma_unscaled,
												homogeneous=False)
    n_guess = A_guess.ncols()
    m_guess = A_guess.nrows()
    (beta_guess, _) = ebdd_guess.estimate_attack()
    dvol_guess = ebdd_guess.dvol
    print("########beta after post + guess########", beta_guess)
    print("scaled? expect <= dim here", ebdd_guess.scaled) #suppose unscaled
    print("ellip_norm: ")
    print(scal((ebdd_guess.u - ebdd_guess.mu) * matrix(np_inv(ebdd_guess.S))/(n_guess + m_guess) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("ellipsoid norm of secret after post + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * ebdd_guess.S.inverse()/(n_guess + m_guess) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("ellipsoid norm of secret after post + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n_guess + m_guess) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("VOL OF ELLIP at Guess Phase: ", ln_vol_ellip(ebdd_guess))
    ####################Intersect Phase#######################
    print("1 ebdd_guess scaled to <= 1?", ebdd_guess.scaled)
    list_av_guess = value_list_update(guess_indices, list_av)
    list_var_guess = value_list_update(guess_indices, list_var)

    intersect_indices_guess = intersect_indices_update(guess_indices, intersect_indices, n) #here n is the original n
    mu_s_intersect, Sigma_s_intersect, Sigma_intersect = hints_integration_Sigma(ebdd_guess, A_guess, b_guess, B_Sigma_guess, list_av_guess, list_var_guess, intersect_indices_guess, guess_indices, m_guess, n_guess, factor, D_s)
    #     mu_s, Sigma_s, Sigma = hints_integration_prior_twist_full_nosubmatrix(ebdd, B_Sigma, list_av, list_var, intersect_indices, eps, n, m, tangent_indices, thres)
    
    (beta_intersect, _) = ebdd_guess.estimate_attack()
    print("2 ebdd_guess scaled to <= 1?", ebdd_guess.scaled)
    dvol_intersect = ebdd_guess.dvol
    print("############beta after integrating SCA info###########", beta_intersect)
    print("expect false: ", ebdd_guess.scaled)
    print("ellipsoid norm of secret after hints_integration: ", scal((ebdd_guess.u - ebdd_guess.mu) * matrix(np_inv(ebdd_guess.S))/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("ellipsoid norm of secret after hints_integration: ", scal((ebdd_guess.u - ebdd_guess.mu) * ebdd_guess.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("VOL OF ELLIP at Intersect Phase: ", ln_vol_ellip(ebdd_guess))
    ###################Short Vector Phase#################
    short_mu_s = ebdd.mu[:, :n]
    short_mu_e = vec([0] * m)
    dbdd_short = switch_to_kannan_embedding(DBDD, A_guess, b_guess, e_vec, q, short_mu_s, short_mu_e, ebdd_guess)
    dbdd_short.integrate_q_vectors(q)#indices=range(n, n + m) 
    (beta_short, _) = dbdd_short.estimate_attack()
    dvol_short = dbdd_short.dvol
    print("########beta after intersect + guess########", beta_short)
#     print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    print("ellip_norm after short vector phase: ")
    print(scal((dbdd_short.u - dbdd_short.mu) * matrix(np_inv(dbdd_short.S))/(dbdd_short.u.ncols()) * ((dbdd_short.u - dbdd_short.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((dbdd_short.u - dbdd_short.mu) * dbdd_short.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("VOL OF ELLIP at Short Vector Phase: ", ln_vol_ellip(dbdd_short))
    
    beta_origin_mean += beta_origin
    beta_post_mean += beta_post
    beta_guess_delete_mean += beta_guess
    beta_intersect_mean += beta_intersect
    beta_short_mean += beta_short
    
    dvol_origin_mean += dvol_origin
    dvol_post_mean += dvol_post
    dvol_guess_delete_mean += dvol_guess
    dvol_intersect_mean += dvol_intersect
    dvol_short_mean += dvol_short
    
    dvol_origin_list[ind] = dvol_origin
    dvol_post_list[ind] = dvol_post
    dvol_guess_delete_list[ind] = dvol_guess
    dvol_intersect_list[ind] = dvol_intersect
    dvol_short_list[ind] = dvol_short
    

    proba_success += p_success
    guesses_total += guesses

#     num_total /= nb_tests_per_param
#     num_guess_total /= nb_tests_per_params
beta_origin_mean /= nb_tests_per_params
beta_post_mean /= nb_tests_per_params
beta_guess_delete_mean /= nb_tests_per_params
beta_intersect_mean /= nb_tests_per_params

dvol_origin_mean /= nb_tests_per_params
dvol_post_mean /= nb_tests_per_params
dvol_guess_delete_mean /= nb_tests_per_params
dvol_intersect_mean /= nb_tests_per_params

proba_success /= nb_tests_per_params
guesses_total /= nb_tests_per_params
logging("number of qualified positions without guess: %3.2f" % num_total, style="HEADER")
logging("number of qualified positions with guess: %3.2f"% num_guess_total, style="HEADER")
logging("Threshold of guesses: %3.2f" %thres_prob, style="HEADER")
logging("Success probability: %3.2f" %proba_success, style="HEADER")
# return list_av, list_var, intersect_indices, ebdd


Sample  0
ellipnorm of secret@initialize:  1.05735930735931


KeyboardInterrupt: 

In [ ]:
#Testing purpose should be syn with def intersect_then_guess
#guess hint then intersect with short vector at the end
num_total = 0
num_guess_total = 0
proba_success = 0
guesses_total = 0

beta_origin_mean = 0
beta_post_mean = 0
beta_guess_delete_mean = 0
beta_intersect_mean = 0
beta_short_mean = 0

dvol_origin_mean = 0
dvol_post_mean = 0
dvol_guess_delete_mean = 0
dvol_intersect_mean = 0
dvol_short_mean = 0

ebdd_list = [0]*nb_tests_per_params
ebdd_guess_list = [0]*nb_tests_per_params

diff_dvol = 0
dvol_origin_list = [0]*nb_tests_per_params
dvol_post_list = [0]*nb_tests_per_params
dvol_guess_delete_list = [0]*nb_tests_per_params
dvol_intersect_list = [0]*nb_tests_per_params
dvol_short_list = [0]*nb_tests_per_params

A2_list = [0]*nb_tests_per_params
b2_list = [0]*nb_tests_per_params
e_vec2_list = [0]*nb_tests_per_params
ebdd2_list = [0]*nb_tests_per_params
B_Sigma2_list = [0]*nb_tests_per_params
measured2_list = [0]*nb_tests_per_params

thres_prob = 0.995
for ind in range(nb_tests_per_params):
    print("Sample ", ind)
    
    ####################Initial Phase#######################
    ###
    
    A = matrix(A2_list[ind])
    b = matrix(b2_list[ind])
    e_vec = matrix(e_vec2_list[ind])
    ebdd = ebdd2_list[ind]
    B_Sigma = matrix(B_Sigma2_list[ind])
    measured = list(measured2_list[ind])
    ###
    
#     A, b, e_vec, ebdd, B_Sigma = initialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo,
#                                                                    n,
#                                                                    q, m, D_s, D_s, homogeneous=False, verbosity=0)
    print("1 scaled to <= 1?", ebdd.scaled)
    (beta_origin, _) = ebdd.estimate_attack()
    dvol_origin =  ebdd.dvol

    print("########initial beta########: ", beta_origin)
    print("1.5 scaled to <= 1?", ebdd.scaled)
    print("VOL OF ELLIP at Initial Phase: ", ln_vol_ellip(ebdd))
    ####################Posteriori Phase#######################
    list_av, list_var, intersect_indices, p_success, guesses, guess_indices, _ = count_SCA_thres(var_prev, measured, thres_prob)

    mu_post, B_post = posteriori_integration_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, n, m, factor)
    (beta_post, _) = ebdd.estimate_attack()
    dvol_post = ebdd.dvol
    print("########beta after only integrating posteriori distribution########", beta_post)
    print("expect false: ", ebdd.scaled)
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * ebdd.S.inverse()/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * B_post * B_post.T/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("VOL OF ELLIP after Posteriori Phase: ", ln_vol_ellip(ebdd))
    ####################Intersect Phase#######################
    
    thres = 0.
    print("1 ebdd scaled to <= 1?", ebdd.scaled)
    Cov_prev = matrix(ebdd.S)
    mu_s_intersect, Sigma_s_intersect, Sigma_intersect = hints_integration_prior_twist_post_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, eps, n, m, factor)
    print("----------------var of guess coordinates----------------")
    Cov_curr = matrix(ebdd.S)
    
    list_cov_prev = []
    list_cov_curr = []
    for i_guess in guess_indices:
#         print(i_guess,list_var[i_guess], Cov_prev[i_guess, i_guess], Cov_curr[i_guess, i_guess])
        if abs(list_var[i_guess] - Cov_prev[i_guess, i_guess]) > 10**(-5):
            print("first two unequal: ", i_guess,list_var[i_guess], Cov_prev[i_guess, i_guess], Cov_curr[i_guess, i_guess])
        if Cov_prev[i_guess, i_guess] < Cov_curr[i_guess, i_guess]:
            print("var is greater after intersect: ", i_guess, Cov_prev[i_guess, i_guess], Cov_curr[i_guess, i_guess])
        
        list_cov_prev += [Cov_prev[i_guess, i_guess]]
        list_cov_curr += [Cov_curr[i_guess, i_guess]]
        
    (beta_intersect, _) = ebdd.estimate_attack()
    print("2 ebdd scaled to <= 1?", ebdd.scaled)
    dvol_intersect = ebdd.dvol

    print("############beta after integrating SCA info###########", beta_intersect)
    print("expect false: ", ebdd.scaled)
    print("ellipsoid norm of secret after hints_integration: ", scal((ebdd.u - ebdd.mu) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("ellipsoid norm of secret after hints_integration: ", scal((ebdd.u - ebdd.mu) * ebdd.S.inverse()/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("VOL OF ELLIP at Intersect Phase: ", ln_vol_ellip(ebdd))
    ####################---Guess Phase---###############
    new_mu_s = ebdd.mu[:, :n]
    new_mu_e = vec([0] * m)
    print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    Sigma_unscaled = ebdd.S  #non-homo, scale to <= dim 
    A_guess, b_guess, e_vec, ebdd_guess, B_Sigma_guess = reinitialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo, A, ebdd.u, e_vec, q, guess_indices, 
												new_mu_s, 
												new_mu_e, 
												Sigma_unscaled,
												homogeneous=False)
    (beta_guess, _) = ebdd_guess.estimate_attack()
    dvol_guess = ebdd_guess.dvol
    print("########beta after intersect + guess########", beta_guess)
    print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    print("ellip_norm: ")
    print(scal((ebdd_guess.u - ebdd_guess.mu) * matrix(np_inv(ebdd_guess.S))/(ebdd_guess.u.ncols()) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * ebdd_guess.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("VOL OF ELLIP at Guess Phase: ", ln_vol_ellip(ebdd_guess))
    ###################Short Vector Phase#################
    dbdd_short = switch_to_kannan_embedding(DBDD, A_guess, b_guess, e_vec, q, new_mu_s, new_mu_e, ebdd_guess)
    dbdd_short.integrate_q_vectors(q)#indices=range(n, n + m) 
    (beta_short, _) = dbdd_short.estimate_attack()
    dvol_short = dbdd_short.dvol
    print("########beta after intersect + guess########", beta_short)
#     print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    print("ellip_norm after short vector phase: ")
    print(scal((dbdd_short.u - dbdd_short.mu) * matrix(np_inv(dbdd_short.S))/(dbdd_short.u.ncols()) * ((dbdd_short.u - dbdd_short.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((dbdd_short.u - dbdd_short.mu) * dbdd_short.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("VOL OF ELLIP at Short Vector Phase: ", ln_vol_ellip(dbdd_short))
    
    
    beta_origin_mean2 += beta_origin
    beta_post_mean2 += beta_post
    beta_guess_delete_mean2 += beta_guess
    beta_intersect_mean2 += beta_intersect
    
    dvol_origin_mean2 += dvol_origin
    dvol_post_mean2 += dvol_post
    dvol_guess_delete_mean2 += dvol_guess
    dvol_intersect_mean2 += dvol_intersect

    dvol_origin_list2[ind] = dvol_origin
    dvol_post_list2[ind] = dvol_post
    dvol_guess_delete_list2[ind] = dvol_guess
    dvol_intersect_list2[ind] = dvol_intersect
    

    proba_success2 += p_success
    guesses_total2 += guesses

#     num_total2 /= nb_tests_per_param
#     num_guess_total /= nb_tests_per_params
beta_origin_mean2 /= nb_tests_per_params
beta_post_mean2 /= nb_tests_per_params
beta_guess_delete_mean2 /= nb_tests_per_params
beta_intersect_mean2 /= nb_tests_per_params

dvol_origin_mean2 /= nb_tests_per_params
dvol_post_mean2 /= nb_tests_per_params
dvol_guess_delete_mean2 /= nb_tests_per_params
dvol_intersect_mean2 /= nb_tests_per_params

proba_success2 /= nb_tests_per_params
guesses_total2 /= nb_tests_per_params
logging("number of qualified positions without guess: %3.2f" % num_total2, style="HEADER")
logging("number of qualified positions with guess: %3.2f"% num_guess_total2, style="HEADER")
logging("Threshold of guesses: %3.2f" %thres_prob, style="HEADER")
logging("Success probability: %3.2f" %proba_success2, style="HEADER")
# return list_av, list_var, intersect_indices, ebdd


In [ ]:
    logging("     intersect_then_guess     ", style="HEADER")
    ####################Initial Phase#######################
    print("1 scaled to <= 1?", ebdd.scaled)
    (beta_origin, _) = ebdd.estimate_attack()
    dvol_origin =  ebdd.dvol

    print("########initial beta########: ", beta_origin)
    print("1.5 scaled to <= 1?", ebdd.scaled)
    print("VOL OF ELLIP at Initial Phase: ", ln_vol_ellip(ebdd))
    ####################Posteriori Phase#######################
    list_av, list_var, intersect_indices, p_success, guesses, guess_indices, _ = count_SCA_thres(var_prev, measured, thres_prob)

    mu_post, B_post = posteriori_integration_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, n, m, factor)
    (beta_post, _) = ebdd.estimate_attack()
    dvol_post = ebdd.dvol
    print("########beta after only integrating posteriori distribution########", beta_post)
    print("expect false: ", ebdd.scaled)
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * ebdd.S.inverse()/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("ellipsoid norm of secret after posteriori_integration: ", scal((ebdd.u - ebdd.mu) * B_post * B_post.T/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("VOL OF ELLIP after Posteriori Phase: ", ln_vol_ellip(ebdd))
    ####################Intersect Phase#######################
    
    thres = 0.
    print("1 ebdd scaled to <= 1?", ebdd.scaled)
    Cov_prev = matrix(ebdd.S)
    mu_s_intersect, Sigma_s_intersect, Sigma_intersect = hints_integration_prior_twist_post_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, eps, n, m, factor)
    print("----------------var of guess coordinates----------------")
    Cov_curr = matrix(ebdd.S)
    
    list_cov_prev = []
    list_cov_curr = []
    for i_guess in guess_indices:
#         print(i_guess,list_var[i_guess], Cov_prev[i_guess, i_guess], Cov_curr[i_guess, i_guess])
        if abs(list_var[i_guess] - Cov_prev[i_guess, i_guess]) > 10**(-5):
            print("first two unequal: ", i_guess,list_var[i_guess], Cov_prev[i_guess, i_guess], Cov_curr[i_guess, i_guess])
        if Cov_prev[i_guess, i_guess] < Cov_curr[i_guess, i_guess]:
            print("var is greater after intersect: ", i_guess, Cov_prev[i_guess, i_guess], Cov_curr[i_guess, i_guess])
        
        list_cov_prev += [Cov_prev[i_guess, i_guess]]
        list_cov_curr += [Cov_curr[i_guess, i_guess]]
        
    (beta_intersect, _) = ebdd.estimate_attack()
    print("2 ebdd scaled to <= 1?", ebdd.scaled)
    dvol_intersect = ebdd.dvol

    print("############beta after integrating SCA info###########", beta_intersect)
    print("expect false: ", ebdd.scaled)
    print("ellipsoid norm of secret after hints_integration: ", scal((ebdd.u - ebdd.mu) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("ellipsoid norm of secret after hints_integration: ", scal((ebdd.u - ebdd.mu) * ebdd.S.inverse()/(n + m) * ((ebdd.u - ebdd.mu)).T))
    print("VOL OF ELLIP at Intersect Phase: ", ln_vol_ellip(ebdd))
    ####################---Guess Phase---###############
    new_mu_s = ebdd.mu[:, :n]
    new_mu_e = vec([0] * m)
    print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    Sigma_unscaled = ebdd.S  #non-homo, scale to <= dim 
    A_guess, b_guess, e_vec, ebdd_guess, B_Sigma_guess = reinitialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo, A, ebdd.u, e_vec, q, guess_indices, 
												new_mu_s, 
												new_mu_e, 
												Sigma_unscaled,
												homogeneous=False)
    (beta_guess, _) = ebdd_guess.estimate_attack()
    dvol_guess = ebdd_guess.dvol
    print("########beta after intersect + guess########", beta_guess)
    print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    print("ellip_norm: ")
    print(scal((ebdd_guess.u - ebdd_guess.mu) * matrix(np_inv(ebdd_guess.S))/(ebdd_guess.u.ncols()) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * ebdd_guess.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("VOL OF ELLIP at Guess Phase: ", ln_vol_ellip(ebdd_guess))
    ###################Short Vector Phase#################
    dbdd_short = switch_to_kannan_embedding(DBDD, A_guess, b_guess, e_vec, q, new_mu_s, new_mu_e, ebdd_guess)
    dbdd_short.integrate_q_vectors(q)#indices=range(n, n + m) 
    (beta_short, _) = dbdd_short.estimate_attack()
    dvol_short = dbdd_short.dvol
    print("########beta after intersect + guess########", beta_short)
#     print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    print("ellip_norm after short vector phase: ")
    print(scal((dbdd_short.u - dbdd_short.mu) * matrix(np_inv(dbdd_short.S))/(dbdd_short.u.ncols()) * ((dbdd_short.u - dbdd_short.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((dbdd_short.u - dbdd_short.mu) * dbdd_short.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    print("VOL OF ELLIP at Short Vector Phase: ", ln_vol_ellip(dbdd_short))
    
    return beta_origin, beta_post, beta_guess, beta_intersect, beta_short, dvol_origin, dvol_post, dvol_guess, dvol_intersect, dvol_short, p_success, guesses, list_cov_prev, list_cov_curr


In [43]:
#guess hint then intersect
num_total = 0
num_guess_total = 0
proba_success = 0
guesses_total = 0

beta_origin_mean = 0
beta_post_mean = 0
beta_guess_delete_mean = 0
beta_intersect_mean = 0

dvol_origin_mean = 0
dvol_post_mean = 0
dvol_guess_delete_mean = 0
dvol_intersect_mean = 0

ebdd_list = [0]*nb_tests_per_params
ebdd_guess_list = [0]*nb_tests_per_params

diff_dvol = 0
dvol_origin_list = [0]*nb_tests_per_params
dvol_post_list = [0]*nb_tests_per_params
dvol_guess_delete_list = [0]*nb_tests_per_params
dvol_intersect_list = [0]*nb_tests_per_params

A2_list = [0]*nb_tests_per_params
b2_list = [0]*nb_tests_per_params
e_vec2_list = [0]*nb_tests_per_params
ebdd2_list = [0]*nb_tests_per_params
B_Sigma2_list = [0]*nb_tests_per_params
measured2_list = [0]*nb_tests_per_params

thres_prob = 0.995
for ind in range(nb_tests_per_params):
    print("Sample ", ind)
    ####################Initial Phase#######################
    A, b, e_vec, ebdd, B_Sigma = initialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo,
                                                                   n,
                                                                   q, m, D_s, D_s, homogeneous=False, verbosity=0)
    ###
    A2 = matrix(A)
    b2 = matrix(b)
    e_vec2 = matrix(e_vec)
    s2 = matrix(ebdd.u[:, :n])
    _, _, _, ebdd2, B_Sigma2 = initialize_kannan_ellipsoid_from_LWE_derandom(EBDD_non_homo,
                                                                              n, q, m, D_s, D_s, 
                                                                              homogeneous=False, verbosity=0,
                                                                              A = A2, s = s2, e_vec = e_vec2)
    A2_list[ind] = A2
    b2_list[ind] = b2
    e_vec2_list[ind] = e_vec2
    ebdd2_list[ind] = ebdd2
    B_Sigma2_list[ind] = B_Sigma2
    
    
    
#     print(b2 == b3)
    
#     print(ebdd.scaled)
#     print(ebdd2.scaled)
    B_Sigma2 = matrix(B_Sigma)
    ###
                                                                          
                                                                          
    print("1 scaled to <= 1?", ebdd.scaled)
    (beta_origin, _) = ebdd.estimate_attack()
    dvol_origin =  ebdd.dvol
    dvol_origin_list[ind] = dvol_origin
    print("########initial beta########: ", beta_origin)
    print("1.5 scaled to <= 1?", ebdd.scaled)
    
    
    ####################Posteriori Phase#######################
    measures = data_to_measure(sca_seeds, params, param, frodo_distribution)
    measured = [simu_measured(ebdd.u[0, i], measures) for i in range(0, n)]
                                                                          
    ###                                                                      
    measured2 = list(measured)
    ###    
    measured2_list[ind] = measured2
                                                                          
    list_av, list_var, intersect_indices, p_success, guesses, guess_indices, _ = count_SCA_thres(var_prev, measured, thres_prob)

#     print("number of useful coordinates: ", len(intersect_indices))
#     print("number of crazy coordinates: ", len(tangent_indices))
#     print("number of perfect guess coordinates: ", len(guess_indices))
#     print("useful coordinates: ", intersect_indices)
#     print("perfect guess coordinate indices: ", guess_indices)
#     print("2 scaled to <= 1?", ebdd.scaled)

    factor = 0
    mu_post, B_post = posteriori_integration_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, n, m, factor)
    (beta_post, _) = ebdd.estimate_attack()
    dvol_post = ebdd.dvol
    print("########beta after only integrating posteriori distribution########", beta_post)
    ebdd_list[ind] = ebdd
#     ####################---Guess Phase---###############
#     new_mu_s = ebdd.mu[:, :n]
#     new_mu_e = vec([0] * m)
#     print("need scaled to <= 1 in guess", ebdd.scaled)
#     Sigma_scaled = ebdd.S * (m + n) #non-homo, scale to <= 1
#     A_guess, b_guess, e_vec, ebdd_guess, B_Sigma_guess = reinitialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo, A, ebdd.u, e_vec, q, guess_indices, 
# 												new_mu_s, 
# 												new_mu_e, 
# 												Sigma_scaled,
# 												homogeneous=False)
#     (beta_guess, _) = ebdd_guess.estimate_attack()
#     dvol_guess = ebdd_guess.dvol
#     print("########beta after posterior + guess########", beta_guess)
    
    ####################---Guess Phase---###############
    new_mu_s = ebdd.mu[:, :n]
    new_mu_e = vec([0] * m)
    print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    Sigma_unscaled = ebdd.S  #non-homo, scale to <= dim 
    A_guess, b_guess, e_vec, ebdd_guess, B_Sigma_guess = reinitialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo, A, ebdd.u, e_vec, q, guess_indices, 
												new_mu_s, 
												new_mu_e, 
												Sigma_unscaled,
												homogeneous=False)
    n_guess = A_guess.ncols()
    m_guess = A_guess.nrows()
    (beta_guess, _) = ebdd_guess.estimate_attack()
    dvol_guess = ebdd_guess.dvol
    print("########beta after intersect + guess########", beta_guess)
    print("ellip_norm: ")
    print(scal((ebdd_guess.u - ebdd_guess.mu) * matrix(np_inv(ebdd_guess.S))/(n_guess + m_guess) * ((ebdd_guess.u - ebdd_guess.mu)).T))

    ####################Intersect Phase#######################
#     thres = 0.
#     n_guess = A_guess.ncols()
#     m_guess = A_guess.nrows()
    print("1 ebdd_guess scaled to <= 1?", ebdd_guess.scaled)
    list_av_guess = value_list_update(guess_indices, list_av)
    list_var_guess = value_list_update(guess_indices, list_var)
    print("in intersect phase n = ", )
    intersect_indices_guess = intersect_indices_update(guess_indices, intersect_indices, n) #here n is the original n
    n_guess1 = n - len(guess_indices)
    print("n_guess compute correctly? ", n_guess1 == n_guess)
#     mu_s_intersect, Sigma_s_intersect, Sigma_intersect = hints_integration_prior_twist_post_extend(ebdd_guess, A_guess, b_guess, B_Sigma_guess, list_av_guess, list_var_guess, intersect_indices_guess, eps, n_guess, m, tangent_indices, thres, factor)
    mu_s_intersect, Sigma_s_intersect, Sigma_intersect = hints_integration_Sigma(ebdd_guess, A_guess, b_guess, B_Sigma_guess, list_av_guess, list_var_guess, intersect_indices_guess, guess_indices, m_guess, n_guess, factor, D_s)
    #     mu_s, Sigma_s, Sigma = hints_integration_prior_twist_full_nosubmatrix(ebdd, B_Sigma, list_av, list_var, intersect_indices, eps, n, m, tangent_indices, thres)
    
    (beta_intersect, _) = ebdd_guess.estimate_attack()
    print("2 ebdd_guess scaled to <= 1?", ebdd_guess.scaled)
    dvol_intersect = ebdd_guess.dvol

    ebdd_guess_list[ind] = ebdd_guess


    print("############beta after integrating SCA info###########", beta_intersect)
    print("5 scaled to <= 1?", ebdd_guess.scaled)

    beta_origin_mean += beta_origin
    beta_post_mean += beta_post
    beta_guess_delete_mean += beta_guess
    beta_intersect_mean += beta_intersect
    
    dvol_origin_mean += dvol_origin
    dvol_post_mean += dvol_post
    dvol_guess_delete_mean += dvol_guess
    dvol_intersect_mean += dvol_intersect

    dvol_origin_list[ind] = dvol_origin
    dvol_post_list[ind] = dvol_post
    dvol_guess_delete_list[ind] = dvol_guess
    dvol_intersect_list[ind] = dvol_intersect
    

    proba_success += p_success
    guesses_total += guesses

#     num_total /= nb_tests_per_param
#     num_guess_total /= nb_tests_per_params
beta_origin_mean /= nb_tests_per_params
beta_post_mean /= nb_tests_per_params
beta_guess_delete_mean /= nb_tests_per_params
beta_intersect_mean /= nb_tests_per_params

dvol_origin_mean /= nb_tests_per_params
dvol_post_mean /= nb_tests_per_params
dvol_guess_delete_mean /= nb_tests_per_params
dvol_intersect_mean /= nb_tests_per_params

proba_success /= nb_tests_per_params
guesses_total /= nb_tests_per_params
logging("number of qualified positions without guess: %3.2f" % num_total, style="HEADER")
logging("number of qualified positions with guess: %3.2f"% num_guess_total, style="HEADER")
logging("Threshold of guesses: %3.2f" %thres_prob, style="HEADER")
logging("Success probability: %3.2f" %proba_success, style="HEADER")
# return list_av, list_var, intersect_indices, ebdd


Sample  0
ellipnorm of secret@initialize:  1.0248917778130533
ellipnorm of secret@initialize:  1.02489177489178
ellipnorm of secret@initialize:  1.0248917778130533
ellipnorm of secret@initialize:  1.02489177489178
1 scaled to <= 1? False
########initial beta########:  269.0772802518226
1.5 scaled to <= 1? False
scaled@funct posteriori_integration?:  False
None in mean_e@mean_s_c_update
1@mean_s_c_update
2@mean_s_c_update
dimension match? 352 352
3@mean_s_c_update
scaled or not suppose false:  False
ellipsoid norm of secret after posteriori_integration:  0.9595629487432333
########beta after only integrating posteriori distribution######## 162.73924621715372
scaled? expect <= dim here False
      Build chopped EBDD from previous LWE      
----------------------
Sigma_unscaled@reinitial:  [  0.000000000000000   0.000000000000000   0.000000000000000   0.000000000000000   0.000000000000000]
[  0.000000000000000   0.106666666666667   0.000000000000000 -0.0400000000000000  0.0466666666666667

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



----------------------
Sigma_s_e@reinitial:  [0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000 0.106666666666667 0.000000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000  1.31333333333333 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000 0.000000000000000  1.31333333333333 0.000000000000000]
[0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000  1.31333333333333] 569
updated Sigma dimension: col, row:  535 535
----------------------
new_Sigma_s_e@reinitial:  [0.0933333333333333  0.000000000000000  0.000000000000000  0.000000000000000  0.000000000000000]
[ 0.000000000000000  0.106666666666667  0.000000000000000  0.000000000000000  0.000000000000000]
[ 0.000000000000000  0.000000000000000   1.31333333333333  0.000000000000000  0.000000000000000]
[ 0.000000000000000  0.000000000000000  0.000000000000000   1.31333333333333  0.000000000000000]
[ 0.000000

/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:426: RuntimeWarning: invalid value encountered in log


(Solution) pi = 0.702920194493547
alpha = 0.259074012712819
X*X_inv MSE = 8.465373340911973e-36
VOL OF ELLIP_OUTPUT:  394.125145756296
dimension match check @hints 183 183 183
dimension match check @hints 352 352 352
None in mean_e@mean_s_c_update
1@mean_s_c_update
2@mean_s_c_update
dimension match? 352 352
3@mean_s_c_update
ellip_norm: 
2.78658879061944
       Attack Estimation      
 dim=535 	 δ=1.009305 	 β=98.98  
  
2 ebdd_guess scaled to <= 1? False
############beta after integrating SCA info########### 98.9834715408472
5 scaled to <= 1? False
 number of qualified positions without guess: 0.00 
 number of qualified positions with guess: 0.00 
 Threshold of guesses: 0.99 
 Success probability: 0.91 


In [92]:
#intersect then remove guess
num_total2 = 0
num_guess_total2 = 0
proba_success2 = 0
guesses_total2 = 0

beta_origin_mean2 = 0
beta_post_mean2 = 0
beta_guess_delete_mean2 = 0
beta_intersect_mean2 = 0

dvol_origin_mean2 = 0
dvol_post_mean2 = 0
dvol_guess_delete_mean2 = 0
dvol_intersect_mean2 = 0

ebdd_list2 = [0]*nb_tests_per_params
ebdd_guess_list2 = [0]*nb_tests_per_params

diff_dvol2 = 0
dvol_origin_list2 = [0]*nb_tests_per_params
dvol_post_list2 = [0]*nb_tests_per_params
dvol_guess_delete_list2 = [0]*nb_tests_per_params
dvol_intersect_list2 = [0]*nb_tests_per_params

thres_prob = 0.98
for ind in range(nb_tests_per_params):
    print("Sample ", ind)
    
    ####################Initial Phase#######################
    ###
    
    A = matrix(A2_list[ind])
    b = matrix(b2_list[ind])
    e_vec = matrix(e_vec2_list[ind])
    ebdd = ebdd2_list[ind]
    B_Sigma = matrix(B_Sigma2_list[ind])
    measured = list(measured2_list[ind])
    ###
    
#     A, b, e_vec, ebdd, B_Sigma = initialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo,
#                                                                    n,
#                                                                    q, m, D_s, D_s, homogeneous=False, verbosity=0)
    print("1 scaled to <= 1?", ebdd.scaled)
    (beta_origin, _) = ebdd.estimate_attack()
    dvol_origin =  ebdd.dvol
    dvol_origin_list2[ind] = dvol_origin
    print("########initial beta########: ", beta_origin)
    print("1.5 scaled to <= 1?", ebdd.scaled)
    
    
    ####################Posteriori Phase#######################
#     measures = data_to_measure(sca_seeds, params, param, frodo_distribution)
#     measured = [simu_measured(ebdd.u[0, i], measures) for i in range(0, n)]
    list_av, list_var, intersect_indices, p_success, guesses, tangent_indices, gap_list, guess_indices = count_SCA_thres(var_prev, measured, thres_prob)

#     print("number of useful coordinates: ", len(intersect_indices))
#     print("number of crazy coordinates: ", len(tangent_indices))
#     print("number of perfect guess coordinates: ", len(guess_indices))
#     print("useful coordinates: ", intersect_indices)
#     print("perfect guess coordinate indices: ", guess_indices)
#     print("2 scaled to <= 1?", ebdd.scaled)

    factor = 0
    mu_post, B_post = posteriori_integration_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, n, m, factor)
    (beta_post, _) = ebdd.estimate_attack()
    dvol_post = ebdd.dvol
    print("########beta after only integrating posteriori distribution########", beta_post)
#     ebdd_list[ind] = ebdd
    
     ####################Intersect Phase#######################
    thres = 0.
    print("1 ebdd scaled to <= 1?", ebdd.scaled)

    mu_s_intersect, Sigma_s_intersect, Sigma_intersect = hints_integration_prior_twist_post_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, eps, n, m, factor)
#     mu_s, Sigma_s, Sigma = hints_integration_prior_twist_full_nosubmatrix(ebdd, B_Sigma, list_av, list_var, intersect_indices, eps, n, m, tangent_indices, thres)
    
    (beta_intersect, _) = ebdd.estimate_attack()
    print("2 ebdd scaled to <= 1?", ebdd.scaled)
    dvol_intersect = ebdd.dvol

    ebdd_list2[ind] = ebdd


    print("############beta after integrating SCA info###########", beta_intersect)
    print("5 scaled to <= 1?", ebdd.scaled)
    
    
    
    ####################---Guess Phase---###############
    new_mu_s = ebdd.mu[:, :n]
    new_mu_e = vec([0] * m)
    print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    Sigma_unscaled = ebdd.S  #non-homo, scale to <= dim 
    A_guess, b_guess, e_vec, ebdd_guess, B_Sigma_guess = reinitialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo, A, ebdd.u, e_vec, q, guess_indices, 
												new_mu_s, 
												new_mu_e, 
												Sigma_unscaled,
												homogeneous=False)
    (beta_guess, _) = ebdd_guess.estimate_attack()
    dvol_guess = ebdd_guess.dvol
    print("########beta after intersect + guess########", beta_guess)
    print("ellip_norm: ")
    print(scal((ebdd_guess.u - ebdd_guess.mu) * matrix(np_inv(ebdd_guess.S))/(ebdd_guess.u.ncols()) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    
    
    ###################Short Vector Phase#################
    dbdd_short = switch_to_kannan_embedding(DBDD, A_guess, b_guess, e_vec, q, new_mu_s, new_mu_e, ebdd_guess)
    dbdd_short.integrate_q_vectors(q)#indices=range(n, n + m) 
    (beta_short, _) = dbdd_short.estimate_attack()
    dvol_short = dbdd_short.dvol
    print("########beta after intersect + guess########", beta_short)
#     print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
    print("ellip_norm after short vector phase: ")
    print(scal((dbdd_short.u - dbdd_short.mu) * matrix(np_inv(dbdd_short.S))/(dbdd_short.u.ncols()) * ((dbdd_short.u - dbdd_short.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((dbdd_short.u - dbdd_short.mu) * dbdd_short.S.inverse()/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
#     print("ellipsoid norm of secret after intersect + guess: ", scal((ebdd_guess.u - ebdd_guess.mu) * B_Sigma_guess * B_Sigma_guess.T/(n + m) * ((ebdd_guess.u - ebdd_guess.mu)).T))
    
    
    beta_origin_mean2 += beta_origin
    beta_post_mean2 += beta_post
    beta_guess_delete_mean2 += beta_guess
    beta_intersect_mean2 += beta_intersect
    
    dvol_origin_mean2 += dvol_origin
    dvol_post_mean2 += dvol_post
    dvol_guess_delete_mean2 += dvol_guess
    dvol_intersect_mean2 += dvol_intersect

    dvol_origin_list2[ind] = dvol_origin
    dvol_post_list2[ind] = dvol_post
    dvol_guess_delete_list2[ind] = dvol_guess
    dvol_intersect_list2[ind] = dvol_intersect
    

    proba_success2 += p_success
    guesses_total2 += guesses

#     num_total2 /= nb_tests_per_param
#     num_guess_total /= nb_tests_per_params
beta_origin_mean2 /= nb_tests_per_params
beta_post_mean2 /= nb_tests_per_params
beta_guess_delete_mean2 /= nb_tests_per_params
beta_intersect_mean2 /= nb_tests_per_params

dvol_origin_mean2 /= nb_tests_per_params
dvol_post_mean2 /= nb_tests_per_params
dvol_guess_delete_mean2 /= nb_tests_per_params
dvol_intersect_mean2 /= nb_tests_per_params

proba_success2 /= nb_tests_per_params
guesses_total2 /= nb_tests_per_params
logging("number of qualified positions without guess: %3.2f" % num_total2, style="HEADER")
logging("number of qualified positions with guess: %3.2f"% num_guess_total2, style="HEADER")
logging("Threshold of guesses: %3.2f" %thres_prob, style="HEADER")
logging("Success probability: %3.2f" %proba_success2, style="HEADER")
# return list_av, list_var, intersect_indices, ebdd


Sample  0
1 scaled to <= 1? False
########initial beta########:  163.05213719674057
1.5 scaled to <= 1? False


ValueError: not enough values to unpack (expected 8, got 7)

In [42]:
ebdd2_list[ind]

[[...]]

In [35]:
mu_s_intersect, Sigma_s_intersect, Sigma_intersect = hints_integration_prior_twist_post_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, eps, n, m, tangent_indices, thres, factor)
#     mu_s, Sigma_s, Sigma = hints_integration_prior_twist_full_nosubmatrix(ebdd, B_Sigma, list_av, list_var, intersect_indices, eps, n, m, tangent_indices, thres)
    
(beta_intersect, _) = ebdd.estimate_attack()
print("2 ebdd scaled to <= 1?", ebdd.scaled)
dvol_intersect = ebdd.dvol

ebdd_list[ind] = ebdd


print("############beta after integrating SCA info###########", beta_intersect)
print("5 scaled to <= 1?", ebdd.scaled)



####################---Guess Phase---###############
new_mu_s = ebdd.mu[:, :n]
new_mu_e = vec([0] * m)
print("scaled? expect <= dim here", ebdd.scaled) #suppose unscaled
Sigma_unscaled = ebdd.S  #non-homo, scale to <= dim 
A_guess, b_guess, e_vec, ebdd_guess, B_Sigma_guess = reinitialize_kannan_ellipsoid_from_LWE_sca(EBDD_non_homo, A, ebdd.u, e_vec, q, guess_indices, 
                                            new_mu_s, 
                                            new_mu_e, 
                                            Sigma_unscaled,
                                            homogeneous=False)
(beta_guess, _) = ebdd_guess.estimate_attack()
dvol_guess = ebdd_guess.dvol
print("########beta after intersect + guess########", beta_guess)
print("ellip_norm: ")
print(scal((ebdd_guess.u - ebdd_guess.mu) * matrix(np_inv(ebdd_guess.S))/(ebdd_guess.u.ncols()) * ((ebdd_guess.u - ebdd_guess.mu)).T))

beta_origin_mean += beta_origin
beta_post_mean += beta_post
beta_guess_delete_mean += beta_guess
beta_intersect_mean += beta_intersect

dvol_origin_mean += dvol_origin
dvol_post_mean += dvol_post
dvol_guess_delete_mean += dvol_guess
dvol_intersect_mean += dvol_intersect

dvol_origin_list[ind] = dvol_origin
dvol_post_list[ind] = dvol_post
dvol_guess_delete_list[ind] = dvol_guess
dvol_intersect_list[ind] = dvol_intersect


proba_success += p_success
guesses_total += guesses

#     num_total /= nb_tests_per_param
#     num_guess_total /= nb_tests_per_params
beta_origin_mean /= nb_tests_per_params
beta_post_mean /= nb_tests_per_params
beta_guess_delete_mean /= nb_tests_per_params
beta_intersect_mean /= nb_tests_per_params

dvol_origin_mean /= nb_tests_per_params
dvol_post_mean /= nb_tests_per_params
dvol_guess_delete_mean /= nb_tests_per_params
dvol_intersect_mean /= nb_tests_per_params

proba_success /= nb_tests_per_params
guesses_total /= nb_tests_per_params
logging("number of qualified positions without guess: %3.2f" % num_total, style="HEADER")
logging("number of qualified positions with guess: %3.2f"% num_guess_total, style="HEADER")
logging("Threshold of guesses: %3.2f" %thres_prob, style="HEADER")
logging("Success probability: %3.2f" %proba_success, style="HEADER")
# return list_av, list_var, intersect_indices, ebdd


----------------------
Bbbbbb@hints:  [0.872871560943970 0.000000000000000  330.818321597765  71.5754679974055  768.126973630693]
[0.000000000000000 0.872871560943970  345.657138133812 -109.108945117996 -58.4823945832460]
[0.000000000000000 0.000000000000000  1787.64095681325 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000 0.000000000000000  1787.64095681325 0.000000000000000]
[0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000  1787.64095681325] 704
uuuuuuu@hints (0, -2, 0, 1, -1, -8, -4, -4, -2, 5)
muuuuuu@hints (0.002646983184478535, -2.184966324829943, 8.2393215487176e-05, 1.210613677231289, -1.1902559124877703, -6.396537407585887, -2.889159174406547, -3.4302436737817086, -0.10049372142957952, 4.829478901818149)
B_Sigma ellipsoid norm:  0.5855052998233887
B_LWE[i, i], c_LWE[0, i] 0.8728715609439696 0.0
B_LWE[i, i], c_LWE[0, i] 0.8728715609439696 0.0
B_LWE[i, i], c_LWE[0, i] 0.8728715609439696 0.0
B_LWE[i, i], c_LWE[0, i] 0.872871560943

/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:153: RuntimeWarning: invalid value encountered in log


(Solution) pi = 0.557496380425316
alpha = 0.685697188426730
VOL OF ELLIP_OUTPUT:  -70.0592160844967
ellip_norm: 
0.802729987416896
2 ebdd scaled to <= 1? False
############beta after integrating SCA info########### 162.2086947013022
5 scaled to <= 1? False
scaled? expect <= dim here False
      Build chopped EBDD from previous LWE      
----------------------
Sigma_unscaled@reinitial:  [   0.220000000000000    0.000000000000000  -0.0400000000000000 -0.00666666666666667  -0.0933333333333333]
[   0.000000000000000    0.373333333333333  -0.0733333333333333   0.0200000000000000   0.0133333333333333]
[ -0.0400000000000000  -0.0733333333333333     2.63333333333333    0.246666666666667   -0.153333333333333]
[-0.00666666666666667   0.0200000000000000    0.246666666666667     2.68666666666667    0.133333333333333]
[ -0.0933333333333333   0.0133333333333333   -0.153333333333333    0.133333333333333     2.48666666666667] 704
----------------------


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



----------------------
Sigma_s_e@reinitial:  [0.220000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000 0.373333333333333 0.000000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000  1.31333333333333 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000 0.000000000000000  1.31333333333333 0.000000000000000]
[0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000  1.31333333333333] 549
updated Sigma dimension: col, row:  501 501
----------------------
new_Sigma_s_e@reinitial:  [0.220000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000 0.373333333333333 0.000000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000  1.31333333333333 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000 0.000000000000000  1.31333333333333 0.000000000000000]
[0.000000000000000 0.000000000

In [15]:
#contains update mean
def hints_integration_prior_twist_post_extend(ebdd, A, b, B_Sigma, list_av, list_var, intersect_indices, eps, n, m, factor):
    """
    Increase LWE ellipsoid in the bad coordinates to ensure the intersections
    B_SCA, c_SCA--posteriori
    B_LWE, c_LWE--prior
    
    """
    print("----------------------")
    print("Bbbbbb@hints: ", matrix(RR,B_Sigma[n - 2 :n + 3,n - 2 :n + 3]), B_Sigma.rank())
    print("uuuuuuu@hints", ebdd.u[0][n - 5 :n + 5])
    print("muuuuuu@hints", ebdd.mu[0][n - 5 :n + 5])
    
    print("B_Sigma ellipsoid norm: ", scal((ebdd.u - ebdd.mu) * B_Sigma * B_Sigma.T/(m + n) * ((ebdd.u - ebdd.mu)).T))
    list_std_inv = [1/sqrt(list_var[i]) for i in range(len(list_var))]
    B_SCA = diagonal_matrix(list_std_inv)
    c_SCA = vec(list_av)
    B_LWE = matrix(B_SCA)
    c_LWE = matrix(c_SCA)

    
    
    
    for i in intersect_indices:
        B_LWE[i, i] = B_Sigma[i][i]
        c_LWE[0, i] = 0
#         print("B_LWE[i, i], c_LWE[0, i]", B_LWE[i, i], c_LWE[0, i])
        

    W_LWE = B_LWE * B_LWE.T * 1/(n)
    W_SCA = B_SCA * B_SCA.T * 1/(n)
    
    c_vec = c_SCA * factor
    sub_denom = scal(c_vec * W_SCA * c_vec.T)
    W_SCA_extend = W_SCA - 1/(1 + sub_denom) * W_SCA * c_vec.T * c_vec * W_SCA
    
    print("VOL OF ELLIP_LWE: ", -ln(W_LWE.det()).n())
    print("VOL OF ELLIP_SCA: ", -ln(W_SCA_extend.det()).n())
    
    mu_s, Sigma_s = ellipsoid_intersection_sca(c_LWE, W_LWE, c_SCA, W_SCA_extend, tolerance=1e-10)
    print("VOL OF ELLIP_OUTPUT: ", ln(Sigma_s.det()).n())
    
    ####update mean#####
    zero1 = zero_matrix(ZZ, m, n)
    upleft_mat = block4(identity_matrix(n), -A.T, zero1, q*identity_matrix(m))
    b_new = b - mu_s * A.T
    b0 = concatenate(zero_matrix(QQ, 1, n), -b_new)
    upleft_mat_inv = upleft_mat.inverse()
    mu = b0 * upleft_mat_inv
    ebdd.mu = concatenate(mu_s, mu[0][n:])
    ####update mean#####
    
#     mu = mean_s_c_update(m, n, mu_s)  
#     ebdd.mu = mu
    
    Sigma_s_inv = matrix(np_inv(Sigma_s * 1/n)) # convert to <= dim
    B_new_sub = matrix(scipy.linalg.cholesky(Sigma_s_inv)) #lower=True

    B_hybrid_eps = block4(B_new_sub, zero_matrix(ZZ, n, m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, m, m))
    E_hybrid_eps = B_hybrid_eps * B_hybrid_eps.T * (1/n) #<= 1
    B_Sigma_eps = block4(zero_matrix(ZZ, n, n), B_Sigma[0:n, n:], B_Sigma[n:, 0:n], B_Sigma[n:, n:]) #<=dim
    
    new_S_inv = (n)/(m + n) * E_hybrid_eps + (1 - (n)/(m + n)) * (1/(m))*B_Sigma_eps*B_Sigma_eps.T    #<= 1          
    ebdd.S = matrix(np_inv(new_S_inv * (n + m))) #<= dim
    print("ellip_norm: ")
    print(scal((ebdd.u - ebdd.mu) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - ebdd.mu)).T))
    return mu_s, Sigma_s, ebdd.S

In [30]:
e_vec2

1 x 352 dense matrix over Integer Ring (use the '.str()' method to see the entries)